# Lab 20: Optimization Algorithms in Bonsai

In this lab, we'll explore the optimization algorithms and techniques that power Bonsai's pedigree reconstruction capabilities. Building on our understanding of Bonsai's architecture, data quality, and multi-sample inference from previous labs, we'll now focus on how Bonsai efficiently searches through the vast space of possible pedigree configurations to find optimal solutions.

## Why This Matters

Pedigree reconstruction is fundamentally a complex optimization problem. The number of possible pedigree configurations grows exponentially with the number of individuals, making exhaustive search impossible for all but the simplest cases. Effective optimization techniques are therefore critical for:
- Efficiently exploring the space of possible pedigrees
- Avoiding getting stuck in local optima
- Balancing multiple competing objectives and constraints
- Scaling to large datasets with hundreds or thousands of individuals
- Providing confidence estimates for the reconstructed relationships

**Learning Objectives**:
- Understand the optimization challenges in pedigree reconstruction
- Implement key optimization algorithms used in Bonsai
- Compare different search strategies for pedigree space exploration
- Develop heuristics to guide the optimization process
- Create visualization tools to track optimization progress
- Apply these techniques to complex pedigree reconstruction problems

## Environment Setup

In [ ]:
import os
import math
import logging
import sys
import re
import warnings
from pathlib import Path
import subprocess
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import networkx as nx
from scipy import stats
from collections import defaultdict, Counter
import random
import time
import json
from tqdm.notebook import tqdm
from dotenv import load_dotenv

## 1. Understanding the Optimization Challenge

Before diving into specific algorithms, let's first understand the nature of the optimization problem in pedigree reconstruction.

### 1.1 The Pedigree Reconstruction Search Space

The search space for pedigree reconstruction is vast and complex:

1. **Size of the search space**: For n individuals, there are 2^(n²) possible pedigree configurations if we consider only whether each pair has a parent-child relationship.

2. **Multiple relationship types**: When we consider different relationship types (sibling, grandparent, cousin, etc.), the space grows even larger.

3. **Constraints**: Valid pedigrees must satisfy numerous biological and logical constraints (e.g., no cycles, consistent birth years, etc.).

4. **Uncertain data**: IBD segments have measurement errors and inference uncertainties that complicate the optimization.

5. **Missing information**: Many real-world pedigrees have missing individuals or incomplete data.

### 1.2 The Objective Function

Bonsai uses a likelihood-based objective function to evaluate pedigree configurations:

1. **IBD likelihood**: How well does the pedigree explain the observed IBD segment data?

2. **Prior probabilities**: Incorporating demographic and historical information as priors.

3. **Constraint penalties**: Penalties for violating biological or logical constraints.

4. **Complexity penalty**: Preference for simpler pedigrees (Occam's razor).

The overall objective is to maximize the posterior probability of the pedigree given the observed data, which balances these components.

### 1.3 Optimization Challenges

Several challenges make this optimization problem particularly difficult:

1. **Non-convexity**: The objective function has many local optima.

2. **Discrete variables**: The relationships are discrete (present/absent), not continuous.

3. **High dimensionality**: Real-world problems involve many individuals and relationships.

4. **Interdependence**: Changing one relationship affects many others.

5. **Complex constraints**: The many constraints on valid pedigrees create a complex feasible region.

In [ ]:
# Visualize the size of the search space
def calculate_search_space_size(n_individuals, n_relationship_types=1):
    """Calculate the size of the pedigree search space.
    
    Args:
        n_individuals: Number of individuals
        n_relationship_types: Number of possible relationship types
        
    Returns:
        Size of the search space
    """
    # For each directed pair, we have n_relationship_types + 1 possibilities (including no relationship)
    directed_pairs = n_individuals * (n_individuals - 1)
    # The size is (n_relationship_types + 1) ^ directed_pairs
    space_size = (n_relationship_types + 1) ** directed_pairs
    return space_size

# Calculate and display search space sizes for different numbers of individuals
individuals_range = range(2, 11)
space_sizes_1_rel = [calculate_search_space_size(n, 1) for n in individuals_range]
space_sizes_3_rel = [calculate_search_space_size(n, 3) for n in individuals_range]

# Create a table to display the results
search_space_df = pd.DataFrame({
    'Number of Individuals': list(individuals_range),
    'Number of Directed Pairs': [n * (n - 1) for n in individuals_range],
    'Search Space Size (1 relationship type)': space_sizes_1_rel,
    'Search Space Size (3 relationship types)': space_sizes_3_rel
})

# Format large numbers
search_space_df['Search Space Size (1 relationship type)'] = search_space_df['Search Space Size (1 relationship type)'].apply(
    lambda x: f"{x:.2e}" if x > 1e10 else f"{x:,}"
)
search_space_df['Search Space Size (3 relationship types)'] = search_space_df['Search Space Size (3 relationship types)'].apply(
    lambda x: f"{x:.2e}" if x > 1e10 else f"{x:,}"
)

# Display the table
display(search_space_df)

# Plot the search space size on a log scale
plt.figure(figsize=(10, 6))
plt.semilogy(individuals_range, space_sizes_1_rel, 'o-', label='1 relationship type')
plt.semilogy(individuals_range, space_sizes_3_rel, 's-', label='3 relationship types')
plt.xlabel('Number of Individuals')
plt.ylabel('Search Space Size (log scale)')
plt.title('Exponential Growth of Pedigree Search Space')
plt.grid(True, alpha=0.3)
plt.legend()
plt.xticks(individuals_range)
plt.tight_layout()
plt.show()

# Calculate some reference points for context
print("For comparison:")
print(f"Number of atoms in the universe: ~10^80")
print(f"Search space size for 10 individuals (3 relationship types): ~{space_sizes_3_rel[-1]:.2e}")

## 2. Core Optimization Algorithms in Bonsai

Bonsai employs several optimization algorithms to navigate the complex search space efficiently. Let's implement and explore these algorithms.

### 2.1 Greedy Incremental Construction

One of the foundational approaches in Bonsai is greedy incremental construction, which builds the pedigree step by step, adding the most confident relationships first.

In [ ]:
# Let's first import the Pedigree class we developed in the previous lab
# This is a simplified version for this notebook

class Pedigree:
    """A class to represent and manipulate pedigree structures."""
    
    def __init__(self):
        """Initialize an empty pedigree."""
        # Use a directed graph to represent parent->child relationships
        self.graph = nx.DiGraph()
        
        # Keep track of individuals by ID
        self.individuals = {}
        
        # Dictionary to track added relationships for quick lookup
        self.relationships = {}
    
    def add_individual(self, id, **attributes):
        """
        Add an individual to the pedigree.
        
        Args:
            id: Unique identifier for the individual
            **attributes: Additional attributes (birth_year, sex, etc.)
            
        Returns:
            bool: True if added successfully, False if individual already exists
        """
        if id in self.individuals:
            return False
        
        # Add to graph
        self.graph.add_node(id, **attributes)
        
        # Store in individuals dictionary
        self.individuals[id] = attributes
        
        return True
    
    def add_relationship(self, parent_id, child_id, certainty=1.0, **attributes):
        """
        Add a parent-child relationship to the pedigree.
        
        Args:
            parent_id: ID of the parent
            child_id: ID of the child
            certainty: Confidence score for this relationship (0.0 to 1.0)
            **attributes: Additional attributes for the relationship
            
        Returns:
            bool: True if added successfully, False otherwise
        """
        # Check if both individuals exist
        if parent_id not in self.individuals or child_id not in self.individuals:
            return False
        
        # Check for impossible relationships (e.g., child as a parent)
        if self.would_create_cycle(parent_id, child_id):
            return False
        
        # Add relationship to graph
        self.graph.add_edge(parent_id, child_id, certainty=certainty, **attributes)
        
        # Store relationship in dictionary for quick lookup
        rel_key = (parent_id, child_id)
        self.relationships[rel_key] = {'certainty': certainty, **attributes}
        
        return True
    
    def would_create_cycle(self, parent_id, child_id):
        """
        Check if adding parent_id as a parent of child_id would create a cycle.
        
        Args:
            parent_id: Proposed parent
            child_id: Proposed child
            
        Returns:
            bool: True if this would create a cycle, False otherwise
        """
        # If child is already an ancestor of parent, this would create a cycle
        try:
            path = nx.shortest_path(self.graph, child_id, parent_id)
            return len(path) > 0
        except nx.NetworkXNoPath:
            return False
    
    def is_consistent(self):
        """
        Check if the pedigree is biologically and logically consistent.
        
        Returns:
            bool: True if consistent, False otherwise
        """
        # Check for cycles (impossible in a real pedigree)
        if not nx.is_directed_acyclic_graph(self.graph):
            return False
        
        # Check birth year consistency (parents should be older than children)
        for parent, child in self.graph.edges():
            if 'birth_year' in self.individuals[parent] and 'birth_year' in self.individuals[child]:
                parent_birth = self.individuals[parent]['birth_year']
                child_birth = self.individuals[child]['birth_year']
                
                if parent_birth >= child_birth:
                    return False
        
        # All checks passed
        return True
    
    def assign_generations(self):
        """
        Assign generation numbers to all individuals in the pedigree.
        
        Generation 0 is assigned to roots (individuals without parents).
        Children are assigned generation numbers incremented from their parents.
        
        Returns:
            dict: Mapping of individual IDs to generation numbers
        """
        # Find roots (individuals without parents)
        roots = [node for node in self.graph.nodes() if self.graph.in_degree(node) == 0]
        
        generations = {}
        
        # Assign generation 0 to roots
        for root in roots:
            generations[root] = 0
        
        # Process all nodes in topological order (parents before children)
        for node in nx.topological_sort(self.graph):
            # If this node is a root, it's already assigned
            if node in generations:
                continue
            
            # Get parents' generations
            parents = list(self.graph.predecessors(node))
            if parents:
                # Assign generation one more than the maximum parent generation
                parent_generations = [generations.get(parent, 0) for parent in parents]
                generations[node] = max(parent_generations) + 1
            else:
                # No parents (but not a root), assign generation 0
                generations[node] = 0
        
        return generations
    
    def visualize(self, highlight_nodes=None, node_labels=True, figsize=(12, 8)):
        """
        Visualize the pedigree.
        
        Args:
            highlight_nodes: List of node IDs to highlight
            node_labels: Whether to show node labels
            figsize: Figure size tuple
            
        Returns:
            matplotlib figure
        """
        # Create figure
        plt.figure(figsize=figsize)
        
        if not self.graph.nodes():
            plt.text(0.5, 0.5, "Empty Pedigree", ha='center', va='center')
            plt.axis('off')
            return plt.gcf()
        
        # Assign generations
        generations = self.assign_generations()
        
        # Create a position layout based on generations (older generations at the top)
        pos = nx.spring_layout(self.graph, seed=42)
        
        # Adjust y position based on generation
        max_gen = max(generations.values()) if generations else 0
        for node in self.graph.nodes():
            gen = generations.get(node, 0)
            # Normalize to 0-1 range with oldest at the top
            norm_gen = 1 - (gen / max(1, max_gen))
            pos[node] = (pos[node][0], 0.8 * norm_gen + 0.1)
        
        # Node colors based on generation
        num_generations = max_gen + 1
        cmap = plt.cm.viridis
        node_colors = [cmap(generations.get(node, 0) / max(1, max_gen)) for node in self.graph.nodes()]
        
        # Draw nodes
        nx.draw_networkx_nodes(self.graph, pos, node_color=node_colors, 
                              node_size=500, alpha=0.8)
        
        # Highlight specific nodes if provided
        if highlight_nodes:
            highlight_nodes = [n for n in highlight_nodes if n in self.graph.nodes()]
            if highlight_nodes:
                nx.draw_networkx_nodes(self.graph.subgraph(highlight_nodes), pos, 
                                      node_color='red', node_size=600, alpha=0.8)
        
        # Draw edges with certainty-based alpha
        edge_alphas = [self.relationships.get((u, v), {}).get('certainty', 1.0) for u, v in self.graph.edges()]
        for i, (u, v) in enumerate(self.graph.edges()):
            # Draw edges with alpha based on certainty
            alpha = edge_alphas[i]
            nx.draw_networkx_edges(self.graph, pos, edgelist=[(u, v)], 
                                 width=1.5, alpha=alpha, arrows=True, 
                                 arrowsize=20, arrowstyle='-|>')
        
        # Add labels
        if node_labels:
            labels = {}
            for node in self.graph.nodes():
                label_parts = [str(node)]
                if 'birth_year' in self.individuals[node]:
                    label_parts.append(f"({self.individuals[node]['birth_year']})")
                labels[node] = "\n".join(label_parts)
            
            nx.draw_networkx_labels(self.graph, pos, labels=labels, font_size=10)
        
        plt.title('Pedigree Visualization', size=15)
        plt.axis('off')
        plt.tight_layout()
        
        return plt.gcf()
    
    def copy(self):
        """Create a deep copy of this pedigree."""
        new_pedigree = Pedigree()
        
        # Copy individuals
        for ind_id, attrs in self.individuals.items():
            new_pedigree.add_individual(ind_id, **attrs.copy())
        
        # Copy relationships
        for parent, child in self.graph.edges():
            attrs = self.graph.edges[parent, child].copy()
            certainty = attrs.pop('certainty', 1.0)
            new_pedigree.add_relationship(parent, child, certainty=certainty, **attrs)
        
        return new_pedigree
    
    def __eq__(self, other):
        """Check if two pedigrees are equal."""
        if not isinstance(other, Pedigree):
            return False
        
        # Check if they have the same individuals
        if set(self.individuals.keys()) != set(other.individuals.keys()):
            return False
        
        # Check if they have the same edges
        if set(self.graph.edges()) != set(other.graph.edges()):
            return False
        
        return True
    
    def score(self, ibd_data, missing_penalty=0.1, extra_penalty=0.2):
        """
        Score the pedigree against IBD data.
        
        Args:
            ibd_data: DataFrame with IBD segment data
            missing_penalty: Penalty for missing relationships
            extra_penalty: Penalty for extra relationships
            
        Returns:
            float: Score (higher is better)
        """
        # Extract pairs with IBD from data
        ibd_pairs = set()
        ibd_values = {}
        
        for _, row in ibd_data.iterrows():
            pair = (row['sample1'], row['sample2'])
            ibd_pairs.add(pair)
            # Store total IBD by pair
            if pair in ibd_values:
                ibd_values[pair] += row['gen_seg_len']
            else:
                ibd_values[pair] = row['gen_seg_len']
        
        # Extract relationships from pedigree
        pedigree_pairs = set()
        for i in self.individuals:
            for j in self.individuals:
                if i == j:
                    continue
                # Check if there's a path from i to j
                try:
                    if nx.has_path(self.graph, i, j) or nx.has_path(self.graph, j, i):
                        pedigree_pairs.add((i, j))
                except:
                    pass
        
        # Calculate score components
        correct_pairs = ibd_pairs.intersection(pedigree_pairs)
        missing_pairs = ibd_pairs - pedigree_pairs
        extra_pairs = pedigree_pairs - ibd_pairs
        
        # Simple scoring model
        score = len(correct_pairs) - missing_penalty * len(missing_pairs) - extra_penalty * len(extra_pairs)
        
        return score

# Now, let's implement the greedy incremental construction algorithm
class GreedyPedigreeBuilder:
    """
    A class for incrementally building pedigrees using a greedy approach.
    """
    
    def __init__(self, segments_df, individuals_metadata=None):
        """
        Initialize the builder with IBD data.
        
        Args:
            segments_df: DataFrame with IBD segments (must have columns: sample1, sample2, gen_seg_len)
            individuals_metadata: Optional DataFrame with metadata about individuals
        """
        self.segments_df = segments_df
        self.individuals_metadata = individuals_metadata
        self.pedigree = Pedigree()
        
        # Initialize by adding all individuals
        self._add_individuals()
        
        # Calculate total IBD sharing between each pair
        self.pair_sharing = self._calculate_pair_sharing()
        
        # Build relationship candidates
        self.candidates = self._build_relationship_candidates()
    
    def _add_individuals(self):
        """Add all individuals from the segment data to the pedigree."""
        # Extract all unique individuals
        all_individuals = set(self.segments_df['sample1']).union(set(self.segments_df['sample2']))
        
        for ind_id in all_individuals:
            # Get metadata for this individual if available
            metadata = {}
            if self.individuals_metadata is not None:
                match = self.individuals_metadata[self.individuals_metadata['id'] == ind_id]
                if len(match) > 0:
                    metadata = match.iloc[0].to_dict()
            
            # Add individual to pedigree
            self.pedigree.add_individual(ind_id, **metadata)
    
    def _calculate_pair_sharing(self):
        """Calculate total IBD sharing for each pair."""
        # Group by pairs and sum segment lengths
        pair_sharing = self.segments_df.groupby(['sample1', 'sample2'])['gen_seg_len'].agg(['sum', 'count']).reset_index()
        return pair_sharing
    
    def _build_relationship_candidates(self):
        """
        Build a list of potential relationship candidates.
        
        Returns:
            list: List of (sample1, sample2, relationship_type, certainty) tuples
        """
        candidates = []
        
        for _, row in self.pair_sharing.iterrows():
            sample1 = row['sample1']
            sample2 = row['sample2']
            total_cm = row['sum']
            
            # Determine relationship type and certainty based on total_cm
            # This is a simplified model and would be more sophisticated in a real implementation
            
            # Parent-child: ~3400 cM
            if 3000 <= total_cm <= 3720:
                rel_type = 'parent-child'
                # Higher certainty for values closer to the expected mean
                certainty = 1.0 - abs(3400 - total_cm) / 400
                candidates.append((sample1, sample2, rel_type, certainty))
            
            # Full sibling: ~2550 cM
            elif 2200 <= total_cm < 3000:
                rel_type = 'full-sibling'
                certainty = 1.0 - abs(2550 - total_cm) / 350
                candidates.append((sample1, sample2, rel_type, certainty))
            
            # Half-sibling/grandparent/avuncular: ~1700 cM
            elif 1450 <= total_cm < 2200:
                # These are ambiguous without additional information
                # We'll add all possibilities with different certainties
                certainty_base = 1.0 - abs(1700 - total_cm) / 250
                
                # Try to disambiguate based on birth years if available
                p1_birth = self.pedigree.individuals[sample1].get('birth_year')
                p2_birth = self.pedigree.individuals[sample2].get('birth_year')
                
                if p1_birth is not None and p2_birth is not None:
                    year_diff = abs(p1_birth - p2_birth)
                    
                    if year_diff < 15:
                        # Likely half-siblings
                        candidates.append((sample1, sample2, 'half-sibling', certainty_base * 0.9))
                    elif 15 <= year_diff <= 30:
                        # Could be avuncular (aunt/uncle - niece/nephew)
                        candidates.append((sample1, sample2, 'avuncular', certainty_base * 0.8))
                    elif year_diff > 30:
                        # Likely grandparent-grandchild
                        # Determine direction based on birth years
                        if p1_birth < p2_birth:
                            candidates.append((sample1, sample2, 'grandparent-grandchild', certainty_base * 0.9))
                        else:
                            candidates.append((sample2, sample1, 'grandparent-grandchild', certainty_base * 0.9))
                else:
                    # Without birth years, add all possibilities with lower certainty
                    candidates.append((sample1, sample2, 'half-sibling', certainty_base * 0.6))
                    candidates.append((sample1, sample2, 'avuncular', certainty_base * 0.5))
                    candidates.append((sample1, sample2, 'grandparent-grandchild', certainty_base * 0.4))
            
            # First cousin: ~850 cM
            elif 550 <= total_cm < 1450:
                rel_type = 'first-cousin'
                certainty = 1.0 - abs(850 - total_cm) / 300
                candidates.append((sample1, sample2, rel_type, certainty))
            
            # More distant relationships with lower certainty
            elif 200 <= total_cm < 550:
                certainty = 0.5
                candidates.append((sample1, sample2, 'distant', certainty))
        
        # Sort candidates by certainty (highest first)
        candidates.sort(key=lambda x: x[3], reverse=True)
        
        return candidates
    
    def build_pedigree(self, max_iterations=None, min_certainty=0.3):
        """
        Build the pedigree using a greedy approach.
        
        Args:
            max_iterations: Maximum number of iterations (relationships to add)
            min_certainty: Minimum certainty threshold for relationships
            
        Returns:
            Pedigree: The constructed pedigree
        """
        # Track metrics
        metrics = {
            'iterations': 0,
            'attempted_relationships': 0,
            'added_relationships': 0,
            'rejected_relationships': 0,
            'certainty_history': []
        }
        
        # Iterate through candidates
        for sample1, sample2, rel_type, certainty in self.candidates:
            # Stop if we've reached the maximum iterations
            if max_iterations is not None and metrics['iterations'] >= max_iterations:
                break
            
            # Skip if certainty is too low
            if certainty < min_certainty:
                continue
            
            metrics['iterations'] += 1
            metrics['attempted_relationships'] += 1
            metrics['certainty_history'].append(certainty)
            
            # Process the relationship based on its type
            if rel_type == 'parent-child':
                # Try to determine which is the parent based on birth years
                p1_birth = self.pedigree.individuals[sample1].get('birth_year')
                p2_birth = self.pedigree.individuals[sample2].get('birth_year')
                
                if p1_birth is not None and p2_birth is not None:
                    if p1_birth < p2_birth:
                        parent, child = sample1, sample2
                    else:
                        parent, child = sample2, sample1
                else:
                    # Without birth years, we can't determine direction
                    # For demonstration, we'll use lexicographic ordering as a placeholder
                    if sample1 < sample2:
                        parent, child = sample1, sample2
                    else:
                        parent, child = sample2, sample1
                
                # Try to add the relationship
                success = self.pedigree.add_relationship(parent, child, certainty=certainty, rel_type=rel_type)
                
                if success:
                    metrics['added_relationships'] += 1
                else:
                    metrics['rejected_relationships'] += 1
            
            elif rel_type == 'full-sibling' or rel_type == 'half-sibling':
                # For siblings, we need to create or find common parents
                # This is a simplified approach
                phantom_parent_id = f"phantom_parent_{sample1}_{sample2}"
                
                # Add phantom parent
                if phantom_parent_id not in self.pedigree.individuals:
                    # Estimate birth year for phantom parent
                    p1_birth = self.pedigree.individuals[sample1].get('birth_year')
                    p2_birth = self.pedigree.individuals[sample2].get('birth_year')
                    
                    parent_birth = None
                    if p1_birth is not None and p2_birth is not None:
                        avg_birth = (p1_birth + p2_birth) // 2
                        parent_birth = avg_birth - 25  # Approximate parent birth year
                    
                    if parent_birth:
                        self.pedigree.add_individual(phantom_parent_id, birth_year=parent_birth, is_phantom=True)
                    else:
                        self.pedigree.add_individual(phantom_parent_id, is_phantom=True)
                
                # Add relationship from phantom parent to both siblings
                success1 = self.pedigree.add_relationship(phantom_parent_id, sample1, certainty=certainty*0.9, rel_type='parent-child')
                success2 = self.pedigree.add_relationship(phantom_parent_id, sample2, certainty=certainty*0.9, rel_type='parent-child')
                
                if success1 and success2:
                    metrics['added_relationships'] += 2
                else:
                    metrics['rejected_relationships'] += 2 - (success1 + success2)
                
                # For full siblings, add a second phantom parent
                if rel_type == 'full-sibling':
                    phantom_parent2_id = f"phantom_parent2_{sample1}_{sample2}"
                    
                    # Add second phantom parent
                    if phantom_parent2_id not in self.pedigree.individuals:
                        # Use similar birth year to first phantom parent
                        parent_birth = self.pedigree.individuals.get(phantom_parent_id, {}).get('birth_year')
                        
                        if parent_birth:
                            self.pedigree.add_individual(phantom_parent2_id, birth_year=parent_birth+2, is_phantom=True)
                        else:
                            self.pedigree.add_individual(phantom_parent2_id, is_phantom=True)
                    
                    # Add relationships from second phantom parent
                    success3 = self.pedigree.add_relationship(phantom_parent2_id, sample1, certainty=certainty*0.8, rel_type='parent-child')
                    success4 = self.pedigree.add_relationship(phantom_parent2_id, sample2, certainty=certainty*0.8, rel_type='parent-child')
                    
                    if success3 and success4:
                        metrics['added_relationships'] += 2
                    else:
                        metrics['rejected_relationships'] += 2 - (success3 + success4)
            
            elif rel_type == 'grandparent-grandchild':
                # For grandparent relationships, we need a phantom parent in between
                phantom_parent_id = f"phantom_parent_{sample1}_{sample2}"
                
                # Determine which is the grandparent based on birth years
                p1_birth = self.pedigree.individuals[sample1].get('birth_year')
                p2_birth = self.pedigree.individuals[sample2].get('birth_year')
                
                if p1_birth is not None and p2_birth is not None:
                    if p1_birth < p2_birth:
                        grandparent, grandchild = sample1, sample2
                    else:
                        grandparent, grandchild = sample2, sample1
                else:
                    # Without birth years, we'll use lexicographic ordering as a placeholder
                    if sample1 < sample2:
                        grandparent, grandchild = sample1, sample2
                    else:
                        grandparent, grandchild = sample2, sample1
                
                # Add phantom parent
                if phantom_parent_id not in self.pedigree.individuals:
                    # Estimate birth year for phantom parent
                    gp_birth = self.pedigree.individuals[grandparent].get('birth_year')
                    gc_birth = self.pedigree.individuals[grandchild].get('birth_year')
                    
                    parent_birth = None
                    if gp_birth is not None and gc_birth is not None:
                        parent_birth = (gp_birth + gc_birth) // 2
                    
                    if parent_birth:
                        self.pedigree.add_individual(phantom_parent_id, birth_year=parent_birth, is_phantom=True)
                    else:
                        self.pedigree.add_individual(phantom_parent_id, is_phantom=True)
                
                # Add relationships
                success1 = self.pedigree.add_relationship(grandparent, phantom_parent_id, certainty=certainty*0.9, rel_type='parent-child')
                success2 = self.pedigree.add_relationship(phantom_parent_id, grandchild, certainty=certainty*0.9, rel_type='parent-child')
                
                if success1 and success2:
                    metrics['added_relationships'] += 2
                else:
                    metrics['rejected_relationships'] += 2 - (success1 + success2)
            
            elif rel_type == 'avuncular':
                # Avuncular relationships (aunt/uncle - niece/nephew) are complex
                # We'll use a simplified approach with a phantom common ancestor
                phantom_id = f"phantom_ancestor_{sample1}_{sample2}"
                
                # Add phantom ancestor
                if phantom_id not in self.pedigree.individuals:
                    # Estimate birth year
                    p1_birth = self.pedigree.individuals[sample1].get('birth_year')
                    p2_birth = self.pedigree.individuals[sample2].get('birth_year')
                    
                    ancestor_birth = None
                    if p1_birth is not None and p2_birth is not None:
                        min_birth = min(p1_birth, p2_birth)
                        ancestor_birth = min_birth - 30
                    
                    if ancestor_birth:
                        self.pedigree.add_individual(phantom_id, birth_year=ancestor_birth, is_phantom=True)
                    else:
                        self.pedigree.add_individual(phantom_id, is_phantom=True)
                
                # Determine likely direction
                if p1_birth is not None and p2_birth is not None:
                    if p1_birth < p2_birth:
                        aunt_uncle, niece_nephew = sample1, sample2
                    else:
                        aunt_uncle, niece_nephew = sample2, sample1
                else:
                    # Without birth years, use lexicographic ordering
                    if sample1 < sample2:
                        aunt_uncle, niece_nephew = sample1, sample2
                    else:
                        aunt_uncle, niece_nephew = sample2, sample1
                
                # Add phantom parent for niece/nephew
                phantom_parent_id = f"phantom_parent_{niece_nephew}"
                
                if phantom_parent_id not in self.pedigree.individuals:
                    # Estimate birth year
                    nn_birth = self.pedigree.individuals[niece_nephew].get('birth_year')
                    anc_birth = self.pedigree.individuals[phantom_id].get('birth_year')
                    
                    parent_birth = None
                    if nn_birth is not None and anc_birth is not None:
                        parent_birth = nn_birth - 25
                    
                    if parent_birth:
                        self.pedigree.add_individual(phantom_parent_id, birth_year=parent_birth, is_phantom=True)
                    else:
                        self.pedigree.add_individual(phantom_parent_id, is_phantom=True)
                
                # Add relationships
                success1 = self.pedigree.add_relationship(phantom_id, aunt_uncle, certainty=certainty*0.8, rel_type='parent-child')
                success2 = self.pedigree.add_relationship(phantom_id, phantom_parent_id, certainty=certainty*0.8, rel_type='parent-child')
                success3 = self.pedigree.add_relationship(phantom_parent_id, niece_nephew, certainty=certainty*0.9, rel_type='parent-child')
                
                if success1 and success2 and success3:
                    metrics['added_relationships'] += 3
                else:
                    metrics['rejected_relationships'] += 3 - (success1 + success2 + success3)
            
            # Other relationship types would be handled similarly
            else:
                # For more distant relationships, we'll skip for this simplified example
                pass
        
        # Return the constructed pedigree
        return self.pedigree, metrics

# Let's create some synthetic IBD data for demonstration
def create_synthetic_ibd_data(num_individuals=15, num_relationships=30):
    """
    Create synthetic IBD data for demonstration purposes.
    
    Args:
        num_individuals: Number of individuals
        num_relationships: Number of relationships to generate
        
    Returns:
        tuple: (segments_df, individuals_df)
    """
    # Create individuals with birth years
    individuals = []
    for i in range(num_individuals):
        birth_year = 1920 + i * 5  # Spread birth years over time
        individuals.append({
            'id': f"ind_{i}",
            'birth_year': birth_year,
            'sex': 'F' if i % 2 == 0 else 'M'  # Alternate sexes
        })
    
    # Create relationships
    segments = []
    for _ in range(num_relationships):
        # Select two random individuals
        i, j = np.random.choice(range(num_individuals), size=2, replace=False)
        ind1, ind2 = f"ind_{i}", f"ind_{j}"
        
        # Determine birth year difference to inform relationship type
        year_diff = abs(individuals[i]['birth_year'] - individuals[j]['birth_year'])
        
        # Assign a relationship type based on birth year difference
        if year_diff > 20 and year_diff < 30:
            # Potential parent-child
            rel_type = 'parent-child'
            target_cm = np.random.normal(3400, 100)
            num_segments_to_generate = np.random.randint(35, 45)
        elif year_diff < 15:
            # Potential siblings or cousins
            if np.random.random() < 0.3:
                rel_type = 'full-sibling'
                target_cm = np.random.normal(2550, 180)
                num_segments_to_generate = np.random.randint(30, 40)
            else:
                rel_type = 'first-cousin'
                target_cm = np.random.normal(850, 150)
                num_segments_to_generate = np.random.randint(15, 25)
        elif year_diff > 40 and year_diff < 60:
            # Potential grandparent-grandchild
            rel_type = 'grandparent-grandchild'
            target_cm = np.random.normal(1700, 160)
            num_segments_to_generate = np.random.randint(20, 30)
        else:
            # Distant relationship
            rel_type = 'distant'
            target_cm = np.random.normal(100, 50)
            num_segments_to_generate = np.random.randint(3, 8)
        
        # Generate segments to approximately match the target cM
        cm_per_segment = target_cm / num_segments_to_generate
        
        for s in range(num_segments_to_generate):
            # Generate segment length (with some variation)
            segment_cm = max(1, np.random.normal(cm_per_segment, cm_per_segment * 0.3))
            
            # Random chromosome
            chrom = np.random.randint(1, 23)
            
            # Add the segment
            segments.append({
                'sample1': ind1,
                'sample2': ind2,
                'chrom': chrom,
                'gen_start': np.random.uniform(0, 200),
                'gen_end': np.random.uniform(0, 200),  # Placeholder
                'gen_seg_len': segment_cm,
                'true_relationship': rel_type  # For evaluation
            })
    
    # Convert to DataFrames
    segments_df = pd.DataFrame(segments)
    individuals_df = pd.DataFrame(individuals)
    
    return segments_df, individuals_df

# Create synthetic data
np.random.seed(42)
synthetic_segments, synthetic_individuals = create_synthetic_ibd_data(num_individuals=10, num_relationships=15)

# Display summary of the data
print(f"Generated {len(synthetic_segments)} segments for {len(synthetic_individuals)} individuals")

# Calculate total sharing per pair
pair_sharing = synthetic_segments.groupby(['sample1', 'sample2', 'true_relationship'])['gen_seg_len'].sum().reset_index()
print("\nSample of total IBD sharing between pairs:")
display(pair_sharing.head(10))

# Now, let's apply the greedy incremental construction algorithm
builder = GreedyPedigreeBuilder(synthetic_segments, synthetic_individuals)

# Build the pedigree
built_pedigree, metrics = builder.build_pedigree(min_certainty=0.3)

# Display metrics
print("\nPedigree construction metrics:")
print(f"Iterations: {metrics['iterations']}")
print(f"Attempted relationships: {metrics['attempted_relationships']}")
print(f"Added relationships: {metrics['added_relationships']}")
print(f"Rejected relationships: {metrics['rejected_relationships']}")

# Visualize the pedigree
built_pedigree.visualize(figsize=(12, 8))

# Plot the certainty history
plt.figure(figsize=(10, 5))
plt.plot(metrics['certainty_history'], 'o-')
plt.xlabel('Iteration')
plt.ylabel('Relationship Certainty')
plt.title('Certainty of Relationships Added During Construction')
plt.grid(True, alpha=0.3)
plt.ylim(0, 1.05)
plt.show()

### 2.2 Simulated Annealing for Pedigree Optimization

While greedy incremental construction is fast, it can get stuck in local optima. Simulated annealing is a more sophisticated approach that allows for temporary "worsening" moves to escape local optima.

In [ ]:
# Let's implement the simulated annealing algorithm for pedigree optimization
class SimulatedAnnealingPedigreeOptimizer:
    """
    A class for optimizing pedigrees using simulated annealing.
    
    Simulated annealing is a probabilistic technique that allows exploration of the search
    space by accepting worse solutions with a probability that decreases over time. This
    helps escape local optima and find better global solutions.
    """
    
    def __init__(self, initial_pedigree, ibd_data, individuals_metadata=None):
        """
        Initialize the optimizer.
        
        Args:
            initial_pedigree: Initial pedigree to start from (can be empty or from greedy construction)
            ibd_data: DataFrame with IBD segments
            individuals_metadata: Optional DataFrame with metadata about individuals
        """
        self.current_pedigree = initial_pedigree.copy()
        self.best_pedigree = initial_pedigree.copy()
        self.ibd_data = ibd_data
        self.individuals_metadata = individuals_metadata
        
        # Set initial scores
        self.current_score = self._score_pedigree(self.current_pedigree)
        self.best_score = self.current_score
        
        # Store all individuals
        self.all_individuals = set(ibd_data['sample1']).union(set(ibd_data['sample2']))
        
        # Calculate IBD sharing between pairs for quick lookup
        self.pair_sharing = self._calculate_pair_sharing()
        
        # Build possible moves
        self.possible_moves = self._build_possible_moves()
    
    def _calculate_pair_sharing(self):
        """Calculate total IBD sharing for each pair."""
        # Group by pairs and sum segment lengths
        return self.ibd_data.groupby(['sample1', 'sample2'])['gen_seg_len'].sum().to_dict()
    
    def _score_pedigree(self, pedigree):
        """
        Calculate a score for the pedigree based on how well it explains the IBD data.
        
        Args:
            pedigree: Pedigree object to score
            
        Returns:
            float: Score (higher is better)
        """
        # We'll use a more comprehensive scoring model for simulated annealing
        
        # 1. Calculate how well pedigree explains IBD segments
        ibd_explanation_score = 0
        
        # Get all pairs in the pedigree with paths
        pedigree_relationships = {}
        for ind1 in pedigree.individuals:
            for ind2 in pedigree.individuals:
                if ind1 == ind2:
                    continue
                    
                # Check if there's a path connecting them
                try:
                    path1 = nx.has_path(pedigree.graph, ind1, ind2)
                    path2 = nx.has_path(pedigree.graph, ind2, ind1)
                    if path1 or path2:
                        # Try to determine relationship degree
                        try:
                            if path1:
                                path_length = len(nx.shortest_path(pedigree.graph, ind1, ind2)) - 1
                            else:
                                path_length = len(nx.shortest_path(pedigree.graph, ind2, ind1)) - 1
                            
                            # Convert path length to expected cM
                            # This is a simplified model; real model would be more sophisticated
                            if path_length == 1:  # Parent-child
                                expected_cm = 3400
                            elif path_length == 2:  # Grandparent or avuncular
                                expected_cm = 1700
                            elif path_length == 3:  # First cousin or great-grandparent
                                expected_cm = 850
                            elif path_length == 4:
                                expected_cm = 425
                            elif path_length == 5:
                                expected_cm = 212
                            else:
                                expected_cm = 850 * (0.5 ** (path_length - 3))
                            
                            pedigree_relationships[(ind1, ind2)] = {
                                'path_length': path_length,
                                'expected_cm': expected_cm
                            }
                        except:
                            # If there's an error calculating the path, use a default
                            pedigree_relationships[(ind1, ind2)] = {
                                'path_length': 0,
                                'expected_cm': 0
                            }
                except:
                    pass
        
        # Compare pedigree relationships with observed IBD
        for pair, expected in pedigree_relationships.items():
            # Check if this pair has observed IBD
            observed_cm = self.pair_sharing.get((pair[0], pair[1]), 0)
            if observed_cm == 0:
                observed_cm = self.pair_sharing.get((pair[1], pair[0]), 0)
            
            # Calculate score contribution (how well does pedigree explain this pair's IBD?)
            # We're using a simple Gaussian model; real implementation would use more complex models
            expected_cm = expected['expected_cm']
            
            # Determine std based on relationship (closer relationships have more variation)
            if expected_cm > 3000:  # Parent-child
                std_cm = 200
            elif expected_cm > 2000:  # Full sibling
                std_cm = 300
            elif expected_cm > 1500:  # Half-sibling/grandparent
                std_cm = 250
            elif expected_cm > 700:  # First cousin
                std_cm = 200
            else:
                std_cm = expected_cm * 0.2  # 20% of expected for distant relationships
            
            # Gaussian score
            score_contrib = stats.norm.pdf(observed_cm, expected_cm, std_cm)
            # Normalize to 0-1 range (roughly)
            score_contrib = score_contrib / stats.norm.pdf(expected_cm, expected_cm, std_cm)
            
            ibd_explanation_score += score_contrib
        
        # 2. Penalty for pairs with IBD but no path in pedigree
        missing_relationship_penalty = 0
        for (ind1, ind2), cm in self.pair_sharing.items():
            if cm < 20:  # Ignore very small segments
                continue
                
            # Check if there's a path in the pedigree
            if (ind1, ind2) not in pedigree_relationships and (ind2, ind1) not in pedigree_relationships:
                # Calculate penalty based on IBD amount (more IBD = higher penalty for missing)
                if cm > 1500:  # Close relationship
                    penalty = 5.0
                elif cm > 700:  # First cousin level
                    penalty = 2.0
                elif cm > 300:  # Second cousin level
                    penalty = 1.0
                elif cm > 100:  # Distant relationship
                    penalty = 0.5
                else:
                    penalty = 0.1
                    
                missing_relationship_penalty += penalty
        
        # 3. Complexity penalty (prefer simpler pedigrees)
        num_edges = pedigree.graph.number_of_edges()
        num_nodes = pedigree.graph.number_of_nodes()
        phantom_nodes = sum(1 for n, attrs in pedigree.individuals.items() if attrs.get('is_phantom', False))
        
        # Penalize complexity, especially phantom nodes
        complexity_penalty = 0.1 * num_edges + 0.5 * phantom_nodes
        
        # 4. Consistency score (reward biologically consistent pedigrees)
        consistency_score = 10 if pedigree.is_consistent() else 0
        
        # Combine all scoring components
        total_score = ibd_explanation_score - missing_relationship_penalty - complexity_penalty + consistency_score
        
        return total_score
    
    def _build_possible_moves(self):
        """
        Build a list of possible moves for modifying the pedigree.
        
        Returns:
            list: List of move types and parameters
        """
        moves = []
        
        # 1. Add relationship between existing individuals
        for ind1 in self.all_individuals:
            for ind2 in self.all_individuals:
                if ind1 == ind2:
                    continue
                
                # Check if there's significant IBD between them
                pair_cm = self.pair_sharing.get((ind1, ind2), 0)
                if pair_cm == 0:
                    pair_cm = self.pair_sharing.get((ind2, ind1), 0)
                
                if pair_cm > 20:  # Only consider pairs with meaningful IBD
                    # For parent-child, we need to determine direction
                    if pair_cm > 2800:  # Potential parent-child
                        # Try to determine direction from birth years
                        ind1_birth = self.current_pedigree.individuals.get(ind1, {}).get('birth_year')
                        ind2_birth = self.current_pedigree.individuals.get(ind2, {}).get('birth_year')
                        
                        if ind1_birth is not None and ind2_birth is not None:
                            if ind1_birth < ind2_birth:
                                moves.append(('add_relationship', ind1, ind2, 'parent-child'))
                            else:
                                moves.append(('add_relationship', ind2, ind1, 'parent-child'))
                        else:
                            # Without birth years, try both directions
                            moves.append(('add_relationship', ind1, ind2, 'parent-child'))
                            moves.append(('add_relationship', ind2, ind1, 'parent-child'))
                    else:
                        # For other relationship types, we'll use phantom nodes in apply_move
                        moves.append(('connect_individuals', ind1, ind2))
        
        # 2. Remove relationship
        for parent, child in self.current_pedigree.graph.edges():
            moves.append(('remove_relationship', parent, child))
        
        # 3. Swap relationships (change parent)
        for child, child_attrs in self.current_pedigree.individuals.items():
            # Get current parents
            parents = list(self.current_pedigree.graph.predecessors(child))
            
            # Potential new parents (anyone older than the child)
            child_birth = child_attrs.get('birth_year')
            potential_parents = []
            
            for ind, attrs in self.current_pedigree.individuals.items():
                if ind == child:
                    continue
                    
                # Check if older (or unknown birth year)
                ind_birth = attrs.get('birth_year')
                if (ind_birth is None or child_birth is None or ind_birth < child_birth) and ind not in parents:
                    potential_parents.append(ind)
            
            # Add swap moves
            for old_parent in parents:
                for new_parent in potential_parents:
                    moves.append(('swap_parent', child, old_parent, new_parent))
        
        # 4. Add/remove phantom node
        for ind1 in self.all_individuals:
            for ind2 in self.all_individuals:
                if ind1 == ind2:
                    continue
                
                # Check if there's significant IBD
                pair_cm = self.pair_sharing.get((ind1, ind2), 0)
                if pair_cm == 0:
                    pair_cm = self.pair_sharing.get((ind2, ind1), 0)
                
                if pair_cm > 200 and pair_cm < 2800:  # Potential distant relationship needing phantom
                    moves.append(('add_phantom_connection', ind1, ind2))
        
        # 5. Move a subtree (e.g., move a whole family branch)
        for node in self.current_pedigree.graph.nodes():
            # Skip if it's a leaf node (no children)
            if self.current_pedigree.graph.out_degree(node) == 0:
                continue
                
            # Get ancestors that could be new parents
            ancestors = []
            for potential_parent in self.current_pedigree.graph.nodes():
                if potential_parent == node:
                    continue
                
                # Check if potential_parent is not a descendant of node
                if not nx.has_path(self.current_pedigree.graph, node, potential_parent):
                    ancestors.append(potential_parent)
            
            # Add moves to connect node to each potential new parent
            for new_parent in ancestors:
                moves.append(('move_subtree', node, new_parent))
        
        return moves
    
    def _apply_move(self, move):
        """
        Apply a move to the current pedigree.
        
        Args:
            move: Tuple describing the move to apply
            
        Returns:
            Pedigree: The modified pedigree
        """
        # Create a copy of the current pedigree
        new_pedigree = self.current_pedigree.copy()
        
        move_type = move[0]
        
        if move_type == 'add_relationship':
            # Add direct relationship between two individuals
            parent, child, rel_type = move[1], move[2], move[3]
            success = new_pedigree.add_relationship(parent, child, rel_type=rel_type)
        
        elif move_type == 'remove_relationship':
            # Remove a relationship
            parent, child = move[1], move[2]
            if new_pedigree.graph.has_edge(parent, child):
                new_pedigree.graph.remove_edge(parent, child)
                # Also remove from relationships dictionary
                if (parent, child) in new_pedigree.relationships:
                    del new_pedigree.relationships[(parent, child)]
                success = True
            else:
                success = False
        
        elif move_type == 'swap_parent':
            # Change a child's parent
            child, old_parent, new_parent = move[1], move[2], move[3]
            
            # Check if both parents exist
            if old_parent in new_pedigree.individuals and new_parent in new_pedigree.individuals:
                # Store old relationship attributes
                old_attrs = new_pedigree.graph.edges.get((old_parent, child), {}).copy()
                
                # Remove old relationship
                if new_pedigree.graph.has_edge(old_parent, child):
                    new_pedigree.graph.remove_edge(old_parent, child)
                    if (old_parent, child) in new_pedigree.relationships:
                        del new_pedigree.relationships[(old_parent, child)]
                
                # Add new relationship with same attributes
                success = new_pedigree.add_relationship(new_parent, child, **old_attrs)
            else:
                success = False
        
        elif move_type == 'connect_individuals':
            # Connect two individuals through appropriate phantom nodes
            ind1, ind2 = move[1], move[2]
            
            # Get IBD amount to determine relationship type
            pair_cm = self.pair_sharing.get((ind1, ind2), 0)
            if pair_cm == 0:
                pair_cm = self.pair_sharing.get((ind2, ind1), 0)
            
            # Determine birth years if available
            ind1_birth = new_pedigree.individuals.get(ind1, {}).get('birth_year')
            ind2_birth = new_pedigree.individuals.get(ind2, {}).get('birth_year')
            
            # Create phantom node ID
            phantom_id = f"phantom_{ind1}_{ind2}"
            
            if 1500 < pair_cm <= 2800:  # Potential siblings or half-siblings
                # Add phantom parent
                if phantom_id not in new_pedigree.individuals:
                    # Estimate birth year for phantom parent
                    parent_birth = None
                    if ind1_birth is not None and ind2_birth is not None:
                        avg_birth = (ind1_birth + ind2_birth) // 2
                        parent_birth = avg_birth - 25  # Approximate parent birth year
                    
                    if parent_birth:
                        new_pedigree.add_individual(phantom_id, birth_year=parent_birth, is_phantom=True)
                    else:
                        new_pedigree.add_individual(phantom_id, is_phantom=True)
                
                # Add relationships to both individuals
                success1 = new_pedigree.add_relationship(phantom_id, ind1, rel_type='parent-child')
                success2 = new_pedigree.add_relationship(phantom_id, ind2, rel_type='parent-child')
                
                success = success1 and success2
            
            elif 700 < pair_cm <= 1500:  # Potential grandparent or avuncular
                # Determine likely relationship direction if birth years available
                if ind1_birth is not None and ind2_birth is not None:
                    year_diff = abs(ind1_birth - ind2_birth)
                    
                    if year_diff > 30:  # Likely grandparent-grandchild
                        if ind1_birth < ind2_birth:
                            older, younger = ind1, ind2
                        else:
                            older, younger = ind2, ind1
                        
                        # Add phantom parent in between
                        phantom_parent_id = f"phantom_parent_{older}_{younger}"
                        
                        # Estimate birth year
                        parent_birth = None
                        if ind1_birth is not None and ind2_birth is not None:
                            parent_birth = (ind1_birth + ind2_birth) // 2
                        
                        if parent_birth:
                            new_pedigree.add_individual(phantom_parent_id, birth_year=parent_birth, is_phantom=True)
                        else:
                            new_pedigree.add_individual(phantom_parent_id, is_phantom=True)
                        
                        # Add relationships
                        success1 = new_pedigree.add_relationship(older, phantom_parent_id, rel_type='parent-child')
                        success2 = new_pedigree.add_relationship(phantom_parent_id, younger, rel_type='parent-child')
                        
                        success = success1 and success2
                    
                    else:  # Likely avuncular (aunt/uncle - niece/nephew)
                        if ind1_birth < ind2_birth:
                            older, younger = ind1, ind2
                        else:
                            older, younger = ind2, ind1
                        
                        # Add phantom common ancestor
                        phantom_ancestor_id = f"phantom_ancestor_{older}_{younger}"
                        
                        # Estimate birth year
                        ancestor_birth = None
                        if older_birth := new_pedigree.individuals.get(older, {}).get('birth_year'):
                            ancestor_birth = older_birth - 25
                        
                        if ancestor_birth:
                            new_pedigree.add_individual(phantom_ancestor_id, birth_year=ancestor_birth, is_phantom=True)
                        else:
                            new_pedigree.add_individual(phantom_ancestor_id, is_phantom=True)
                        
                        # Add phantom parent for younger
                        phantom_parent_id = f"phantom_parent_{younger}"
                        
                        # Estimate birth year
                        parent_birth = None
                        if younger_birth := new_pedigree.individuals.get(younger, {}).get('birth_year'):
                            parent_birth = younger_birth - 25
                        
                        if parent_birth:
                            new_pedigree.add_individual(phantom_parent_id, birth_year=parent_birth, is_phantom=True)
                        else:
                            new_pedigree.add_individual(phantom_parent_id, is_phantom=True)
                        
                        # Add relationships
                        success1 = new_pedigree.add_relationship(phantom_ancestor_id, older, rel_type='parent-child')
                        success2 = new_pedigree.add_relationship(phantom_ancestor_id, phantom_parent_id, rel_type='parent-child')
                        success3 = new_pedigree.add_relationship(phantom_parent_id, younger, rel_type='parent-child')
                        
                        success = success1 and success2 and success3
                else:
                    # Without birth years, use a generic approach (just connect through a phantom)
                    if phantom_id not in new_pedigree.individuals:
                        new_pedigree.add_individual(phantom_id, is_phantom=True)
                    
                    success1 = new_pedigree.add_relationship(phantom_id, ind1, rel_type='connection')
                    success2 = new_pedigree.add_relationship(phantom_id, ind2, rel_type='connection')
                    
                    success = success1 and success2
            
            elif 200 < pair_cm <= 700:  # Potential cousins
                # Add phantom common ancestor
                if phantom_id not in new_pedigree.individuals:
                    # Estimate birth year (if available)
                    ancestor_birth = None
                    if ind1_birth is not None and ind2_birth is not None:
                        min_birth = min(ind1_birth, ind2_birth)
                        ancestor_birth = min_birth - 50  # Approx. for common ancestor of cousins
                    
                    if ancestor_birth:
                        new_pedigree.add_individual(phantom_id, birth_year=ancestor_birth, is_phantom=True)
                    else:
                        new_pedigree.add_individual(phantom_id, is_phantom=True)
                
                # Add phantom parents
                phantom_parent1_id = f"phantom_parent_{ind1}"
                phantom_parent2_id = f"phantom_parent_{ind2}"
                
                # Estimate birth years (if available)
                parent1_birth = None
                parent2_birth = None
                if ind1_birth is not None:
                    parent1_birth = ind1_birth - 25
                if ind2_birth is not None:
                    parent2_birth = ind2_birth - 25
                
                # Add phantom parents
                if phantom_parent1_id not in new_pedigree.individuals:
                    if parent1_birth:
                        new_pedigree.add_individual(phantom_parent1_id, birth_year=parent1_birth, is_phantom=True)
                    else:
                        new_pedigree.add_individual(phantom_parent1_id, is_phantom=True)
                
                if phantom_parent2_id not in new_pedigree.individuals:
                    if parent2_birth:
                        new_pedigree.add_individual(phantom_parent2_id, birth_year=parent2_birth, is_phantom=True)
                    else:
                        new_pedigree.add_individual(phantom_parent2_id, is_phantom=True)
                
                # Add relationships
                success1 = new_pedigree.add_relationship(phantom_id, phantom_parent1_id, rel_type='parent-child')
                success2 = new_pedigree.add_relationship(phantom_id, phantom_parent2_id, rel_type='parent-child')
                success3 = new_pedigree.add_relationship(phantom_parent1_id, ind1, rel_type='parent-child')
                success4 = new_pedigree.add_relationship(phantom_parent2_id, ind2, rel_type='parent-child')
                
                success = success1 and success2 and success3 and success4
            
            else:  # Very distant or unsupported relationship
                success = False
        
        elif move_type == 'add_phantom_connection':
            # Add a phantom node to connect two individuals
            ind1, ind2 = move[1], move[2]
            phantom_id = f"phantom_connector_{ind1}_{ind2}"
            
            # Create phantom node
            if phantom_id not in new_pedigree.individuals:
                new_pedigree.add_individual(phantom_id, is_phantom=True)
            
            # Connect both individuals to phantom
            success1 = new_pedigree.add_relationship(phantom_id, ind1, rel_type='connection')
            success2 = new_pedigree.add_relationship(phantom_id, ind2, rel_type='connection')
            
            success = success1 and success2
        
        elif move_type == 'move_subtree':
            # Move a subtree by changing the parent of the subtree root
            subtree_root, new_parent = move[1], move[2]
            
            # Get current parents
            current_parents = list(new_pedigree.graph.predecessors(subtree_root))
            
            # Check if this would create a cycle
            if new_pedigree.would_create_cycle(new_parent, subtree_root):
                success = False
            else:
                # Remove current parent connections
                for parent in current_parents:
                    if new_pedigree.graph.has_edge(parent, subtree_root):
                        new_pedigree.graph.remove_edge(parent, subtree_root)
                        if (parent, subtree_root) in new_pedigree.relationships:
                            del new_pedigree.relationships[(parent, subtree_root)]
                
                # Add new parent connection
                success = new_pedigree.add_relationship(new_parent, subtree_root, rel_type='parent-child')
        
        else:
            # Unknown move type
            success = False
        
        return new_pedigree if success else None
    
    def _get_random_move(self):
        """Get a random move from the possible moves list."""
        if not self.possible_moves:
            return None
        return random.choice(self.possible_moves)
    
    def _acceptance_probability(self, old_score, new_score, temperature):
        """
        Calculate the probability of accepting a move that changes the score.
        
        Args:
            old_score: Score before the move
            new_score: Score after the move
            temperature: Current temperature (controls acceptance probability)
            
        Returns:
            float: Probability of accepting the move (0-1)
        """
        # If the new solution is better, accept it with probability 1
        if new_score > old_score:
            return 1.0
        
        # If the new solution is worse, calculate acceptance probability
        # The higher the temperature, the more likely we are to accept a worse solution
        try:
            return math.exp((new_score - old_score) / temperature)
        except OverflowError:
            # Handle very large exponents
            return 0.0 if new_score < old_score else 1.0
    
    def optimize(self, max_iterations=1000, initial_temp=10.0, cooling_rate=0.003, min_temp=0.01):
        """
        Run the simulated annealing optimization process.
        
        Args:
            max_iterations: Maximum number of iterations
            initial_temp: Initial temperature
            cooling_rate: Rate at which temperature decreases
            min_temp: Minimum temperature to stop at
            
        Returns:
            tuple: (best_pedigree, metrics)
        """
        # Initialize tracking variables
        current_temp = initial_temp
        iteration = 0
        
        # Metrics to track progress
        metrics = {
            'iterations': 0,
            'attempted_moves': 0,
            'accepted_moves': 0,
            'rejected_moves': 0,
            'temperature_history': [],
            'score_history': [],
            'best_score_history': []
        }
        
        # Main simulated annealing loop
        progress_bar = tqdm(total=max_iterations, desc="Optimizing Pedigree")
        
        while iteration < max_iterations and current_temp > min_temp:
            # Update metrics
            metrics['iterations'] = iteration
            metrics['temperature_history'].append(current_temp)
            metrics['score_history'].append(self.current_score)
            metrics['best_score_history'].append(self.best_score)
            
            # Get a random move
            move = self._get_random_move()
            if move is None:
                break
            
            # Apply the move
            metrics['attempted_moves'] += 1
            new_pedigree = self._apply_move(move)
            
            # Skip if the move was invalid
            if new_pedigree is None:
                continue
            
            # Score the new pedigree
            new_score = self._score_pedigree(new_pedigree)
            
            # Decide whether to accept the move
            acceptance_prob = self._acceptance_probability(self.current_score, new_score, current_temp)
            if random.random() < acceptance_prob:
                # Accept the move
                self.current_pedigree = new_pedigree
                self.current_score = new_score
                metrics['accepted_moves'] += 1
                
                # Update best pedigree if this is the best we've seen
                if new_score > self.best_score:
                    self.best_pedigree = new_pedigree.copy()
                    self.best_score = new_score
            else:
                # Reject the move
                metrics['rejected_moves'] += 1
            
            # Cool the temperature
            current_temp *= (1 - cooling_rate)
            iteration += 1
            
            # Update progress bar
            progress_bar.update(1)
            
            # Update possible moves occasionally to reflect the changed pedigree
            if iteration % 50 == 0:
                self.possible_moves = self._build_possible_moves()
        
        progress_bar.close()
        
        # Final update to metrics
        metrics['iterations'] = iteration
        metrics['temperature_history'].append(current_temp)
        metrics['score_history'].append(self.current_score)
        metrics['best_score_history'].append(self.best_score)
        
        return self.best_pedigree, metrics

In [ ]:
# Test the simulated annealing implementation with the synthetic data

# First, build an initial pedigree using the greedy algorithm
greedy_builder = GreedyPedigreeBuilder(synthetic_segments, synthetic_individuals)
initial_pedigree, greedy_metrics = greedy_builder.build_pedigree(min_certainty=0.3)

# Now, optimize it using simulated annealing
optimizer = SimulatedAnnealingPedigreeOptimizer(initial_pedigree, synthetic_segments, synthetic_individuals)

# Run the optimization
best_pedigree, sa_metrics = optimizer.optimize(
    max_iterations=200,  # Reduced for notebook demonstration
    initial_temp=10.0,
    cooling_rate=0.01,
    min_temp=0.1
)

# Display metrics
print("\nSimulated Annealing Metrics:")
print(f"Iterations: {sa_metrics['iterations']}")
print(f"Attempted moves: {sa_metrics['attempted_moves']}")
print(f"Accepted moves: {sa_metrics['accepted_moves']}")
print(f"Rejected moves: {sa_metrics['rejected_moves']}")
print(f"Initial score: {sa_metrics['score_history'][0]}")
print(f"Final score: {sa_metrics['score_history'][-1]}")
print(f"Best score: {sa_metrics['best_score_history'][-1]}")

# Compare greedy and optimized pedigrees
print("\nComparison:")
print(f"Greedy pedigree score: {optimizer._score_pedigree(initial_pedigree)}")
print(f"Optimized pedigree score: {optimizer._score_pedigree(best_pedigree)}")
print(f"Improvement: {100 * (optimizer._score_pedigree(best_pedigree) - optimizer._score_pedigree(initial_pedigree)) / abs(optimizer._score_pedigree(initial_pedigree)):.2f}%")

# Plot temperature and score history
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))

# Temperature plot
ax1.plot(sa_metrics['temperature_history'], 'r-')
ax1.set_xlabel('Iteration')
ax1.set_ylabel('Temperature')
ax1.set_title('Temperature Decay During Simulated Annealing')
ax1.grid(True, alpha=0.3)

# Score plot
ax2.plot(sa_metrics['score_history'], 'b-', alpha=0.5, label='Current Score')
ax2.plot(sa_metrics['best_score_history'], 'g-', linewidth=2, label='Best Score')
ax2.set_xlabel('Iteration')
ax2.set_ylabel('Score')
ax2.set_title('Score Evolution During Simulated Annealing')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Visualize the optimized pedigree
plt.figure(figsize=(12, 8))
best_pedigree.visualize()
plt.title('Optimized Pedigree from Simulated Annealing')
plt.show()

# Visualize the initial (greedy) pedigree for comparison
plt.figure(figsize=(12, 8))
initial_pedigree.visualize()
plt.title('Initial Pedigree from Greedy Algorithm')
plt.show()

### 2.3 Genetic Algorithms for Pedigree Optimization

Genetic algorithms provide another optimization approach, inspired by biological evolution. This method maintains a population of candidate pedigrees and evolves them through selection, crossover, and mutation to find optimal solutions.

In [ ]:
# Let's implement a genetic algorithm for pedigree optimization
class GeneticPedigreeOptimizer:
    """
    A class for optimizing pedigrees using genetic algorithms.
    
    Genetic algorithms maintain a population of candidate solutions (pedigrees) and
    evolve them through selection, crossover, and mutation operations, mimicking
    natural selection and evolution.
    """
    
    def __init__(self, ibd_data, individuals_metadata=None, population_size=20):
        """
        Initialize the optimizer.
        
        Args:
            ibd_data: DataFrame with IBD segments
            individuals_metadata: Optional DataFrame with metadata about individuals
            population_size: Size of the population to maintain
        """
        self.ibd_data = ibd_data
        self.individuals_metadata = individuals_metadata
        self.population_size = population_size
        
        # Store all individuals
        self.all_individuals = set(ibd_data['sample1']).union(set(ibd_data['sample2']))
        
        # Calculate IBD sharing between pairs for quick lookup
        self.pair_sharing = self._calculate_pair_sharing()
        
        # Initialize population
        self.population = self._initialize_population()
        
        # Track the best solution
        self.best_pedigree = self.population[0]
        self.best_score = self._score_pedigree(self.best_pedigree)
    
    def _calculate_pair_sharing(self):
        """Calculate total IBD sharing for each pair."""
        # Group by pairs and sum segment lengths
        return self.ibd_data.groupby(['sample1', 'sample2'])['gen_seg_len'].sum().to_dict()
    
    def _initialize_population(self):
        """
        Create the initial population of pedigrees.
        
        Returns:
            list: List of Pedigree objects
        """
        population = []
        
        # Create some greedy pedigrees with different parameters
        greedy_certainties = [0.2, 0.3, 0.5, 0.7]
        for certainty in greedy_certainties:
            # Create a greedy pedigree with this certainty threshold
            builder = GreedyPedigreeBuilder(self.ibd_data, self.individuals_metadata)
            pedigree, _ = builder.build_pedigree(min_certainty=certainty)
            population.append(pedigree)
        
        # Create some random pedigrees
        for _ in range(self.population_size - len(greedy_certainties)):
            # Create a random pedigree
            pedigree = self._create_random_pedigree()
            population.append(pedigree)
        
        # Evaluate all pedigrees
        population_with_scores = [(p, self._score_pedigree(p)) for p in population]
        
        # Sort by score
        population_with_scores.sort(key=lambda x: x[1], reverse=True)
        
        # Update best pedigree
        if population_with_scores:
            self.best_pedigree = population_with_scores[0][0].copy()
            self.best_score = population_with_scores[0][1]
        
        # Return sorted population
        return [p for p, _ in population_with_scores]
    
    def _create_random_pedigree(self):
        """
        Create a random pedigree.
        
        Returns:
            Pedigree: A randomly generated pedigree
        """
        pedigree = Pedigree()
        
        # Add all individuals
        for ind_id in self.all_individuals:
            # Get metadata if available
            metadata = {}
            if self.individuals_metadata is not None:
                match = self.individuals_metadata[self.individuals_metadata['id'] == ind_id]
                if len(match) > 0:
                    metadata = match.iloc[0].to_dict()
            
            pedigree.add_individual(ind_id, **metadata)
        
        # Add some random relationships based on IBD sharing
        for (ind1, ind2), cm in self.pair_sharing.items():
            if cm < 20:  # Ignore very small segments
                continue
                
            # Add relationships with higher probability for higher IBD
            probability = min(0.8, cm / 3500)  # Cap at 80% even for very high IBD
            
            if random.random() < probability:
                # Determine relationship type based on cM
                if cm > 2800:  # Likely parent-child
                    # Try to determine direction
                    ind1_birth = pedigree.individuals.get(ind1, {}).get('birth_year')
                    ind2_birth = pedigree.individuals.get(ind2, {}).get('birth_year')
                    
                    if ind1_birth is not None and ind2_birth is not None:
                        if ind1_birth < ind2_birth:
                            parent, child = ind1, ind2
                        else:
                            parent, child = ind2, ind1
                    else:
                        # Without birth years, use random direction
                        if random.random() < 0.5:
                            parent, child = ind1, ind2
                        else:
                            parent, child = ind2, ind1
                    
                    # Check if adding this relationship would be valid
                    if not pedigree.would_create_cycle(parent, child):
                        pedigree.add_relationship(parent, child, certainty=probability, rel_type='parent-child')
                    
                elif cm > 1500:  # Siblings or half-siblings
                    # For siblings, we need a common parent
                    phantom_parent_id = f"phantom_parent_{ind1}_{ind2}"
                    
                    # Add phantom parent
                    if phantom_parent_id not in pedigree.individuals:
                        # Estimate birth year if possible
                        p1_birth = pedigree.individuals.get(ind1, {}).get('birth_year')
                        p2_birth = pedigree.individuals.get(ind2, {}).get('birth_year')
                        
                        parent_birth = None
                        if p1_birth is not None and p2_birth is not None:
                            avg_birth = (p1_birth + p2_birth) // 2
                            parent_birth = avg_birth - 25  # Approximate parent birth year
                        
                        if parent_birth:
                            pedigree.add_individual(phantom_parent_id, birth_year=parent_birth, is_phantom=True)
                        else:
                            pedigree.add_individual(phantom_parent_id, is_phantom=True)
                    
                    # Add relationships
                    pedigree.add_relationship(phantom_parent_id, ind1, certainty=probability, rel_type='parent-child')
                    pedigree.add_relationship(phantom_parent_id, ind2, certainty=probability, rel_type='parent-child')
                
                elif cm > 700:  # Cousins or avuncular
                    # For these relationships, we'll add a suitable phantom structure
                    if random.random() < 0.5:  # Avuncular
                        phantom_parent_id = f"phantom_parent_{ind1}_{ind2}"
                        
                        # Add phantom parent
                        if phantom_parent_id not in pedigree.individuals:
                            pedigree.add_individual(phantom_parent_id, is_phantom=True)
                        
                        # Determine direction randomly
                        if random.random() < 0.5:
                            pedigree.add_relationship(phantom_parent_id, ind1, certainty=probability, rel_type='parent-child')
                            pedigree.add_relationship(ind1, ind2, certainty=probability, rel_type='avuncular')
                        else:
                            pedigree.add_relationship(phantom_parent_id, ind2, certainty=probability, rel_type='parent-child')
                            pedigree.add_relationship(ind2, ind1, certainty=probability, rel_type='avuncular')
                    
                    else:  # First cousins
                        phantom_grandparent_id = f"phantom_grandparent_{ind1}_{ind2}"
                        phantom_parent1_id = f"phantom_parent1_{ind1}_{ind2}"
                        phantom_parent2_id = f"phantom_parent2_{ind1}_{ind2}"
                        
                        # Add phantom individuals
                        if phantom_grandparent_id not in pedigree.individuals:
                            pedigree.add_individual(phantom_grandparent_id, is_phantom=True)
                        if phantom_parent1_id not in pedigree.individuals:
                            pedigree.add_individual(phantom_parent1_id, is_phantom=True)
                        if phantom_parent2_id not in pedigree.individuals:
                            pedigree.add_individual(phantom_parent2_id, is_phantom=True)
                        
                        # Add relationships
                        pedigree.add_relationship(phantom_grandparent_id, phantom_parent1_id, certainty=probability, rel_type='parent-child')
                        pedigree.add_relationship(phantom_grandparent_id, phantom_parent2_id, certainty=probability, rel_type='parent-child')
                        pedigree.add_relationship(phantom_parent1_id, ind1, certainty=probability, rel_type='parent-child')
                        pedigree.add_relationship(phantom_parent2_id, ind2, certainty=probability, rel_type='parent-child')
        
        return pedigree
    
    def _score_pedigree(self, pedigree):
        """
        Calculate a score for the pedigree.
        
        Args:
            pedigree: Pedigree object to score
            
        Returns:
            float: Score (higher is better)
        """
        # This is the same scoring function used in SimulatedAnnealingPedigreeOptimizer
        # We'll use a simplified version here for clarity
        
        # 1. Calculate how well pedigree explains IBD segments
        ibd_explanation_score = 0
        
        # Get all pairs in the pedigree with paths
        pedigree_relationships = {}
        for ind1 in pedigree.individuals:
            for ind2 in pedigree.individuals:
                if ind1 == ind2:
                    continue
                    
                # Check if there's a path connecting them
                try:
                    path1 = nx.has_path(pedigree.graph, ind1, ind2)
                    path2 = nx.has_path(pedigree.graph, ind2, ind1)
                    if path1 or path2:
                        # Try to determine relationship degree
                        try:
                            if path1:
                                path_length = len(nx.shortest_path(pedigree.graph, ind1, ind2)) - 1
                            else:
                                path_length = len(nx.shortest_path(pedigree.graph, ind2, ind1)) - 1
                            
                            # Convert path length to expected cM
                            if path_length == 1:  # Parent-child
                                expected_cm = 3400
                            elif path_length == 2:  # Grandparent or avuncular
                                expected_cm = 1700
                            elif path_length == 3:  # First cousin or great-grandparent
                                expected_cm = 850
                            elif path_length == 4:
                                expected_cm = 425
                            elif path_length == 5:
                                expected_cm = 212
                            else:
                                expected_cm = 850 * (0.5 ** (path_length - 3))
                            
                            pedigree_relationships[(ind1, ind2)] = {
                                'path_length': path_length,
                                'expected_cm': expected_cm
                            }
                        except:
                            # If there's an error calculating the path, use a default
                            pedigree_relationships[(ind1, ind2)] = {
                                'path_length': 0,
                                'expected_cm': 0
                            }
                except:
                    pass
        
        # Compare pedigree relationships with observed IBD
        for pair, expected in pedigree_relationships.items():
            # Check if this pair has observed IBD
            observed_cm = self.pair_sharing.get((pair[0], pair[1]), 0)
            if observed_cm == 0:
                observed_cm = self.pair_sharing.get((pair[1], pair[0]), 0)
            
            # Calculate score contribution
            expected_cm = expected['expected_cm']
            
            # Determine std based on relationship
            if expected_cm > 3000:  # Parent-child
                std_cm = 200
            elif expected_cm > 2000:  # Full sibling
                std_cm = 300
            elif expected_cm > 1500:  # Half-sibling/grandparent
                std_cm = 250
            elif expected_cm > 700:  # First cousin
                std_cm = 200
            else:
                std_cm = expected_cm * 0.2
            
            # Gaussian score
            score_contrib = stats.norm.pdf(observed_cm, expected_cm, std_cm)
            # Normalize to 0-1 range
            score_contrib = score_contrib / stats.norm.pdf(expected_cm, expected_cm, std_cm)
            
            ibd_explanation_score += score_contrib
        
        # 2. Penalty for pairs with IBD but no path in pedigree
        missing_relationship_penalty = 0
        for (ind1, ind2), cm in self.pair_sharing.items():
            if cm < 20:  # Ignore very small segments
                continue
                
            # Check if there's a path in the pedigree
            if (ind1, ind2) not in pedigree_relationships and (ind2, ind1) not in pedigree_relationships:
                # Calculate penalty based on IBD amount
                if cm > 1500:  # Close relationship
                    penalty = 5.0
                elif cm > 700:  # First cousin level
                    penalty = 2.0
                elif cm > 300:  # Second cousin level
                    penalty = 1.0
                elif cm > 100:  # Distant relationship
                    penalty = 0.5
                else:
                    penalty = 0.1
                    
                missing_relationship_penalty += penalty
        
        # 3. Complexity penalty
        num_edges = pedigree.graph.number_of_edges()
        phantom_nodes = sum(1 for n, attrs in pedigree.individuals.items() if attrs.get('is_phantom', False))
        
        # Penalize complexity
        complexity_penalty = 0.1 * num_edges + 0.5 * phantom_nodes
        
        # 4. Consistency score
        consistency_score = 10 if pedigree.is_consistent() else 0
        
        # Combine all scoring components
        total_score = ibd_explanation_score - missing_relationship_penalty - complexity_penalty + consistency_score
        
        return total_score
    
    def _select_parent(self, scores):
        """
        Select a parent for crossover using tournament selection.
        
        Args:
            scores: List of scores for each pedigree
            
        Returns:
            int: Index of the selected parent
        """
        # Tournament selection
        # Select k individuals at random and return the best
        k = 3  # Tournament size
        indices = random.sample(range(len(scores)), k)
        best_idx = max(indices, key=lambda i: scores[i])
        return best_idx
    
    def _crossover(self, parent1, parent2):
        """
        Create a child pedigree by combining elements from two parents.
        
        Args:
            parent1: First parent pedigree
            parent2: Second parent pedigree
            
        Returns:
            Pedigree: Child pedigree
        """
        # Create a new empty pedigree
        child = Pedigree()
        
        # Add all individuals from both parents
        all_individuals = set(parent1.individuals.keys()).union(set(parent2.individuals.keys()))
        for ind_id in all_individuals:
            # Get attributes from either parent
            attrs = {}
            if ind_id in parent1.individuals:
                attrs = parent1.individuals[ind_id].copy()
            elif ind_id in parent2.individuals:
                attrs = parent2.individuals[ind_id].copy()
            
            # Add individual to child
            child.add_individual(ind_id, **attrs)
        
        # Cross over relationships
        # We'll take some relationships from parent1 and some from parent2
        
        # Get all edges from both parents
        all_edges = set(parent1.graph.edges()).union(set(parent2.graph.edges()))
        
        # Sort edges to ensure deterministic behavior
        all_edges = sorted(all_edges)
        
        # For each edge, randomly choose whether to include it in the child
        for parent, child_id in all_edges:
            # Skip if either node doesn't exist in the child
            if parent not in child.individuals or child_id not in child.individuals:
                continue
                
            # Get relationship attributes from the appropriate parent
            attrs = {}
            if (parent, child_id) in parent1.graph.edges():
                attrs = parent1.graph.edges[parent, child_id].copy()
            elif (parent, child_id) in parent2.graph.edges():
                attrs = parent2.graph.edges[parent, child_id].copy()
            
            # Decide whether to add this relationship to the child
            if random.random() < 0.5:
                # Check if this would create a cycle (skip if it would)
                if not child.would_create_cycle(parent, child_id):
                    child.add_relationship(parent, child_id, **attrs)
        
        return child
    
    def _mutate(self, pedigree, mutation_rate=0.1):
        """
        Apply random mutations to a pedigree.
        
        Args:
            pedigree: Pedigree to mutate
            mutation_rate: Probability of each mutation type
            
        Returns:
            Pedigree: Mutated pedigree
        """
        # Create a copy of the pedigree
        mutated = pedigree.copy()
        
        # 1. Randomly remove some relationships
        for parent, child in list(mutated.graph.edges()):  # Create a list to avoid modifying during iteration
            if random.random() < mutation_rate:
                mutated.graph.remove_edge(parent, child)
                if (parent, child) in mutated.relationships:
                    del mutated.relationships[(parent, child)]
        
        # 2. Randomly add some relationships
        # For each pair with significant IBD
        for (ind1, ind2), cm in self.pair_sharing.items():
            if cm < 100:  # Only consider pairs with meaningful IBD
                continue
                
            if random.random() < mutation_rate * (cm / 3500):  # Higher probability for higher IBD
                # Determine relationship type based on cM
                if cm > 2800:  # Likely parent-child
                    # Try to determine direction
                    ind1_birth = mutated.individuals.get(ind1, {}).get('birth_year')
                    ind2_birth = mutated.individuals.get(ind2, {}).get('birth_year')
                    
                    if ind1_birth is not None and ind2_birth is not None:
                        if ind1_birth < ind2_birth:
                            parent, child = ind1, ind2
                        else:
                            parent, child = ind2, ind1
                    else:
                        # Without birth years, use random direction
                        if random.random() < 0.5:
                            parent, child = ind1, ind2
                        else:
                            parent, child = ind2, ind1
                    
                    # Check if adding this relationship would be valid
                    if not mutated.would_create_cycle(parent, child):
                        mutated.add_relationship(parent, child, rel_type='parent-child')
        
        # 3. Randomly add phantom nodes to connect individuals
        if random.random() < mutation_rate:
            # Select two random individuals with IBD but no connection
            disconnected_pairs = []
            for (ind1, ind2), cm in self.pair_sharing.items():
                if cm < 100:  # Only consider pairs with meaningful IBD
                    continue
                    
                # Check if there's a path in the pedigree
                try:
                    path1 = nx.has_path(mutated.graph, ind1, ind2)
                    path2 = nx.has_path(mutated.graph, ind2, ind1)
                    if not (path1 or path2):
                        disconnected_pairs.append((ind1, ind2, cm))
                except:
                    disconnected_pairs.append((ind1, ind2, cm))
            
            if disconnected_pairs:
                # Select a random pair, with higher probability for higher IBD
                weights = [cm for _, _, cm in disconnected_pairs]
                total_weight = sum(weights)
                if total_weight > 0:
                    r = random.uniform(0, total_weight)
                    cum_weight = 0
                    for idx, (ind1, ind2, cm) in enumerate(disconnected_pairs):
                        cum_weight += cm
                        if cum_weight >= r:
                            selected_pair = (ind1, ind2)
                            break
                    else:
                        selected_pair = disconnected_pairs[-1][:2]
                    
                    # Add a phantom node to connect them
                    phantom_id = f"phantom_mutation_{ind1}_{ind2}"
                    
                    # Add phantom node
                    if phantom_id not in mutated.individuals:
                        mutated.add_individual(phantom_id, is_phantom=True)
                    
                    # Add relationships
                    mutated.add_relationship(phantom_id, ind1, rel_type='connection')
                    mutated.add_relationship(phantom_id, ind2, rel_type='connection')
        
        return mutated
    
    def optimize(self, generations=50, mutation_rate=0.1):
        """
        Run the genetic algorithm optimization process.
        
        Args:
            generations: Number of generations to evolve
            mutation_rate: Probability of mutation
            
        Returns:
            tuple: (best_pedigree, metrics)
        """
        # Initialize metrics
        metrics = {
            'generations': 0,
            'population_size': self.population_size,
            'mean_score_history': [],
            'max_score_history': [],
            'best_score': self.best_score
        }
        
        # Main loop
        progress_bar = tqdm(total=generations, desc="Evolving Pedigrees")
        
        for generation in range(generations):
            # Evaluate all pedigrees
            scores = [self._score_pedigree(p) for p in self.population]
            
            # Update metrics
            mean_score = sum(scores) / len(scores)
            max_score = max(scores)
            metrics['mean_score_history'].append(mean_score)
            metrics['max_score_history'].append(max_score)
            
            # Update best pedigree if improved
            best_idx = scores.index(max_score)
            if max_score > self.best_score:
                self.best_pedigree = self.population[best_idx].copy()
                self.best_score = max_score
                metrics['best_score'] = max_score
            
            # Create next generation
            next_generation = []
            
            # Elitism: keep the best individual
            next_generation.append(self.population[best_idx].copy())
            
            # Create the rest through crossover and mutation
            while len(next_generation) < self.population_size:
                # Select parents
                parent1_idx = self._select_parent(scores)
                parent2_idx = self._select_parent(scores)
                
                # Create child through crossover
                child = self._crossover(self.population[parent1_idx], self.population[parent2_idx])
                
                # Apply mutation
                if random.random() < mutation_rate:
                    child = self._mutate(child, mutation_rate)
                
                # Add to next generation
                next_generation.append(child)
            
            # Replace population
            self.population = next_generation
            
            # Update progress bar
            progress_bar.update(1)
        
        progress_bar.close()
        
        # Final update to metrics
        metrics['generations'] = generations
        
        return self.best_pedigree, metrics

# Test the genetic algorithm with the synthetic data
# Initialize the optimizer
ga_optimizer = GeneticPedigreeOptimizer(synthetic_segments, synthetic_individuals, population_size=10)

# Run the optimization
ga_best_pedigree, ga_metrics = ga_optimizer.optimize(generations=20, mutation_rate=0.2)

# Display metrics
print("\nGenetic Algorithm Metrics:")
print(f"Generations: {ga_metrics['generations']}")
print(f"Population size: {ga_metrics['population_size']}")
print(f"Initial best score: {ga_metrics['max_score_history'][0]}")
print(f"Final best score: {ga_metrics['max_score_history'][-1]}")
print(f"Overall best score: {ga_metrics['best_score']}")

# Compare with previous methods
sa_score = optimizer._score_pedigree(best_pedigree)
ga_score = optimizer._score_pedigree(ga_best_pedigree)
greedy_score = optimizer._score_pedigree(initial_pedigree)

print("\nComparison of All Methods:")
print(f"Greedy algorithm score: {greedy_score}")
print(f"Simulated annealing score: {sa_score}")
print(f"Genetic algorithm score: {ga_score}")

# Plot evolution of scores during genetic algorithm
plt.figure(figsize=(10, 6))
plt.plot(ga_metrics['mean_score_history'], 'b-', label='Mean Score')
plt.plot(ga_metrics['max_score_history'], 'r-', label='Max Score')
plt.xlabel('Generation')
plt.ylabel('Score')
plt.title('Evolution of Pedigree Scores in Genetic Algorithm')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Visualize the genetic algorithm's best pedigree
plt.figure(figsize=(12, 8))
ga_best_pedigree.visualize()
plt.title('Best Pedigree from Genetic Algorithm')
plt.show()

<cell_type>markdown</cell_type>## 3. Comparing Optimization Approaches

Now that we've implemented and tested three different optimization approaches (greedy incremental construction, simulated annealing, and genetic algorithms), let's compare their effectiveness, efficiency, and suitability for different pedigree reconstruction scenarios.

In [ ]:
# Let's systematically compare our three optimization approaches
# Create a function to run all three methods on the same data and compare results

def compare_optimization_methods(ibd_data, individuals_metadata, 
                               greedy_min_certainty=0.3,
                               sa_iterations=200, sa_initial_temp=10.0, sa_cooling_rate=0.01, sa_min_temp=0.1,
                               ga_population_size=10, ga_generations=20, ga_mutation_rate=0.2):
    """
    Compare the three optimization methods on the same dataset.
    
    Args:
        ibd_data: DataFrame with IBD segments
        individuals_metadata: DataFrame with individual metadata
        greedy_min_certainty: Minimum certainty for greedy algorithm
        sa_iterations, sa_initial_temp, sa_cooling_rate, sa_min_temp: Simulated annealing parameters
        ga_population_size, ga_generations, ga_mutation_rate: Genetic algorithm parameters
        
    Returns:
        tuple: (results_df, best_pedigrees)
    """
    results = []
    best_pedigrees = {}
    
    # Run greedy algorithm
    print("Running greedy incremental construction...")
    start_time = time.time()
    greedy_builder = GreedyPedigreeBuilder(ibd_data, individuals_metadata)
    greedy_pedigree, greedy_metrics = greedy_builder.build_pedigree(min_certainty=greedy_min_certainty)
    greedy_time = time.time() - start_time
    
    # Score greedy pedigree
    scorer = SimulatedAnnealingPedigreeOptimizer(greedy_pedigree, ibd_data, individuals_metadata)
    greedy_score = scorer._score_pedigree(greedy_pedigree)
    
    # Save results
    results.append({
        'Method': 'Greedy Incremental Construction',
        'Score': greedy_score,
        'Time (s)': greedy_time,
        'Num Individuals': len(greedy_pedigree.individuals),
        'Num Relationships': greedy_pedigree.graph.number_of_edges(),
        'Num Phantom Nodes': sum(1 for _, attrs in greedy_pedigree.individuals.items() if attrs.get('is_phantom', False))
    })
    best_pedigrees['Greedy'] = greedy_pedigree
    
    # Run simulated annealing
    print("Running simulated annealing...")
    start_time = time.time()
    sa_optimizer = SimulatedAnnealingPedigreeOptimizer(greedy_pedigree, ibd_data, individuals_metadata)
    sa_pedigree, sa_metrics = sa_optimizer.optimize(
        max_iterations=sa_iterations,
        initial_temp=sa_initial_temp,
        cooling_rate=sa_cooling_rate,
        min_temp=sa_min_temp
    )
    sa_time = time.time() - start_time
    sa_score = scorer._score_pedigree(sa_pedigree)
    
    # Save results
    results.append({
        'Method': 'Simulated Annealing',
        'Score': sa_score,
        'Time (s)': sa_time,
        'Num Individuals': len(sa_pedigree.individuals),
        'Num Relationships': sa_pedigree.graph.number_of_edges(),
        'Num Phantom Nodes': sum(1 for _, attrs in sa_pedigree.individuals.items() if attrs.get('is_phantom', False))
    })
    best_pedigrees['Simulated Annealing'] = sa_pedigree
    
    # Run genetic algorithm
    print("Running genetic algorithm...")
    start_time = time.time()
    ga_optimizer = GeneticPedigreeOptimizer(ibd_data, individuals_metadata, population_size=ga_population_size)
    ga_pedigree, ga_metrics = ga_optimizer.optimize(generations=ga_generations, mutation_rate=ga_mutation_rate)
    ga_time = time.time() - start_time
    ga_score = scorer._score_pedigree(ga_pedigree)
    
    # Save results
    results.append({
        'Method': 'Genetic Algorithm',
        'Score': ga_score,
        'Time (s)': ga_time,
        'Num Individuals': len(ga_pedigree.individuals),
        'Num Relationships': ga_pedigree.graph.number_of_edges(),
        'Num Phantom Nodes': sum(1 for _, attrs in ga_pedigree.individuals.items() if attrs.get('is_phantom', False))
    })
    best_pedigrees['Genetic Algorithm'] = ga_pedigree
    
    # Create DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df, best_pedigrees

# Run the comparison on our synthetic data
print("Comparing optimization methods on synthetic data...")
comparison_results, best_pedigrees = compare_optimization_methods(
    synthetic_segments, synthetic_individuals,
    greedy_min_certainty=0.3,
    sa_iterations=150,  # Reduced for demonstration
    sa_initial_temp=10.0,
    sa_cooling_rate=0.01,
    sa_min_temp=0.1,
    ga_population_size=8,  # Reduced for demonstration
    ga_generations=15,    # Reduced for demonstration
    ga_mutation_rate=0.2
)

# Display results
print("\nOptimization Method Comparison:")
display(comparison_results)

# Create a bar chart comparing the methods
plt.figure(figsize=(10, 6))
comparison_results.set_index('Method')['Score'].plot(kind='bar', color=['blue', 'green', 'orange'])
plt.title('Pedigree Score by Optimization Method')
plt.ylabel('Score')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# Create a bar chart comparing execution time
plt.figure(figsize=(10, 6))
comparison_results.set_index('Method')['Time (s)'].plot(kind='bar', color=['blue', 'green', 'orange'])
plt.title('Execution Time by Optimization Method')
plt.ylabel('Time (seconds)')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# Create a grouped bar chart for structural comparison
comparison_results_melted = pd.melt(
    comparison_results, 
    id_vars=['Method'], 
    value_vars=['Num Individuals', 'Num Relationships', 'Num Phantom Nodes'],
    var_name='Metric', 
    value_name='Count'
)

plt.figure(figsize=(12, 6))
sns.barplot(data=comparison_results_melted, x='Method', y='Count', hue='Metric')
plt.title('Structural Comparison of Pedigrees')
plt.xlabel('')
plt.xticks(rotation=45)
plt.legend(title='')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# Display the best pedigree from each method for visual comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for i, (method, pedigree) in enumerate(best_pedigrees.items()):
    plt.sca(axes[i])
    pedigree.visualize()
    plt.title(f'Best Pedigree: {method}')
    plt.tight_layout()

plt.show()

<cell_type>markdown</cell_type>### 3.1 Optimization Algorithm Tradeoffs

Let's analyze the tradeoffs between our three optimization approaches:

1. **Greedy Incremental Construction**
   - **Advantages**: Fast, simple to implement, good for initializing other methods
   - **Disadvantages**: Gets stuck in local optima, order-dependent, doesn't handle conflicting evidence well
   - **Best for**: Small pedigrees, well-separated relationships, initial exploration, generating starting points for other optimization methods

2. **Simulated Annealing**
   - **Advantages**: Can escape local optima, flexible move operators, single sequence to track/debug
   - **Disadvantages**: Parameter tuning required (cooling schedule), slower than greedy
   - **Best for**: Medium-sized pedigrees with complex relationships, refining greedy solutions

3. **Genetic Algorithms**
   - **Advantages**: Explores multiple solutions in parallel, good for large search spaces, resistant to getting stuck
   - **Disadvantages**: Implementation complexity, resource intensive, difficult to engineer effective crossover
   - **Best for**: Large pedigrees, exploring diverse solution topologies, problems with many local optima

In real-world applications, Bonsai often uses these methods in combination:
1. Begin with greedy construction to build initial pedigree 
2. Refine with simulated annealing for better local structure
3. Use genetic algorithms for exploring alternative global structures

<cell_type>markdown</cell_type>### 3.2 Hybrid Approach

Let's implement a hybrid optimization approach that combines the strengths of each method:

In [ ]:
# Let's implement a hybrid optimization approach
class HybridPedigreeOptimizer:
    """
    A hybrid optimization approach for pedigree reconstruction that combines
    greedy construction, simulated annealing, and genetic algorithms.
    """
    
    def __init__(self, ibd_data, individuals_metadata=None):
        """
        Initialize the optimizer.
        
        Args:
            ibd_data: DataFrame with IBD segments
            individuals_metadata: Optional DataFrame with metadata about individuals
        """
        self.ibd_data = ibd_data
        self.individuals_metadata = individuals_metadata
        
        # Store the best pedigree and score
        self.best_pedigree = None
        self.best_score = float('-inf')
    
    def optimize(self, greedy_min_certainties=None, 
                 sa_iterations=100, sa_cooling_rates=None, 
                 ga_population_size=10, ga_generations=15):
        """
        Run the hybrid optimization process.
        
        Args:
            greedy_min_certainties: List of certainty thresholds for greedy construction
            sa_iterations: Number of iterations for simulated annealing
            sa_cooling_rates: List of cooling rates for simulated annealing
            ga_population_size: Size of the population for genetic algorithm
            ga_generations: Number of generations for genetic algorithm
            
        Returns:
            tuple: (best_pedigree, metrics)
        """
        # Default parameter values
        if greedy_min_certainties is None:
            greedy_min_certainties = [0.3, 0.4, 0.5, 0.6]
        
        if sa_cooling_rates is None:
            sa_cooling_rates = [0.005, 0.01, 0.02]
        
        # Initialize metrics
        metrics = {
            'greedy_pedigrees': [],
            'sa_pedigrees': [],
            'ga_best_pedigree': None,
            'score_progression': [],
            'time_greedy': 0,
            'time_sa': 0,
            'time_ga': 0,
            'total_time': 0
        }
        
        start_total_time = time.time()
        
        # Step 1: Generate different greedy pedigrees with varying certainty thresholds
        print("Step 1: Generating greedy pedigrees with different certainty thresholds...")
        start_time = time.time()
        
        greedy_pedigrees = []
        for min_certainty in greedy_min_certainties:
            print(f"  Building greedy pedigree with min_certainty={min_certainty}...")
            builder = GreedyPedigreeBuilder(self.ibd_data, self.individuals_metadata)
            pedigree, _ = builder.build_pedigree(min_certainty=min_certainty)
            greedy_pedigrees.append(pedigree)
            
            # Score the pedigree
            scorer = SimulatedAnnealingPedigreeOptimizer(pedigree, self.ibd_data, self.individuals_metadata)
            score = scorer._score_pedigree(pedigree)
            
            # Update best if improved
            if score > self.best_score:
                self.best_pedigree = pedigree.copy()
                self.best_score = score
            
            # Update metrics
            metrics['greedy_pedigrees'].append({
                'min_certainty': min_certainty,
                'score': score,
                'num_relationships': pedigree.graph.number_of_edges()
            })
            metrics['score_progression'].append({
                'step': 'greedy',
                'parameter': min_certainty,
                'score': score
            })
        
        metrics['time_greedy'] = time.time() - start_time
        
        # Step 2: Apply simulated annealing to each greedy pedigree with different cooling rates
        print("\nStep 2: Refining with simulated annealing using different cooling rates...")
        start_time = time.time()
        
        sa_pedigrees = []
        for i, pedigree in enumerate(greedy_pedigrees):
            for cooling_rate in sa_cooling_rates:
                print(f"  Running SA on greedy pedigree {i+1} with cooling_rate={cooling_rate}...")
                sa_optimizer = SimulatedAnnealingPedigreeOptimizer(pedigree, self.ibd_data, self.individuals_metadata)
                sa_pedigree, _ = sa_optimizer.optimize(
                    max_iterations=sa_iterations,
                    initial_temp=10.0,
                    cooling_rate=cooling_rate,
                    min_temp=0.1
                )
                sa_pedigrees.append(sa_pedigree)
                
                # Score the pedigree
                scorer = SimulatedAnnealingPedigreeOptimizer(sa_pedigree, self.ibd_data, self.individuals_metadata)
                score = scorer._score_pedigree(sa_pedigree)
                
                # Update best if improved
                if score > self.best_score:
                    self.best_pedigree = sa_pedigree.copy()
                    self.best_score = score
                
                # Update metrics
                metrics['sa_pedigrees'].append({
                    'greedy_index': i,
                    'cooling_rate': cooling_rate,
                    'score': score,
                    'num_relationships': sa_pedigree.graph.number_of_edges()
                })
                metrics['score_progression'].append({
                    'step': 'simulated_annealing',
                    'parameter': cooling_rate,
                    'score': score
                })
        
        metrics['time_sa'] = time.time() - start_time
        
        # Step 3: Use genetic algorithm with the best pedigrees so far as part of the initial population
        print("\nStep 3: Running genetic algorithm with best pedigrees as seed population...")
        start_time = time.time()
        
        # Collect all pedigrees from previous steps
        all_pedigrees = greedy_pedigrees + sa_pedigrees
        
        # Sort by score
        pedigree_scores = []
        for p in all_pedigrees:
            scorer = SimulatedAnnealingPedigreeOptimizer(p, self.ibd_data, self.individuals_metadata)
            pedigree_scores.append((p, scorer._score_pedigree(p)))
        
        pedigree_scores.sort(key=lambda x: x[1], reverse=True)
        
        # Take the top pedigrees as initial population for GA
        initial_population = [p for p, _ in pedigree_scores[:min(ga_population_size, len(pedigree_scores))]]
        
        # If we need more for the population, create random ones
        while len(initial_population) < ga_population_size:
            print(f"  Adding random pedigree to initial GA population...")
            ga_optimizer = GeneticPedigreeOptimizer(self.ibd_data, self.individuals_metadata, population_size=1)
            random_pedigree = ga_optimizer._create_random_pedigree()
            initial_population.append(random_pedigree)
        
        # Create custom GA optimizer that starts with our population
        class SeededGeneticPedigreeOptimizer(GeneticPedigreeOptimizer):
            def __init__(self, ibd_data, initial_population, individuals_metadata=None):
                self.ibd_data = ibd_data
                self.individuals_metadata = individuals_metadata
                self.population_size = len(initial_population)
                
                # Use the provided initial population
                self.population = initial_population
                
                # Store all individuals
                self.all_individuals = set(ibd_data['sample1']).union(set(ibd_data['sample2']))
                
                # Calculate IBD sharing between pairs for quick lookup
                self.pair_sharing = self._calculate_pair_sharing()
                
                # Evaluate the population
                population_with_scores = [(p, self._score_pedigree(p)) for p in self.population]
                population_with_scores.sort(key=lambda x: x[1], reverse=True)
                
                # Track the best solution
                self.best_pedigree = population_with_scores[0][0].copy()
                self.best_score = population_with_scores[0][1]
        
        # Run the genetic algorithm with the seeded population
        print(f"  Running GA with population_size={ga_population_size}, generations={ga_generations}...")
        ga_optimizer = SeededGeneticPedigreeOptimizer(self.ibd_data, initial_population, self.individuals_metadata)
        ga_pedigree, ga_metrics = ga_optimizer.optimize(generations=ga_generations, mutation_rate=0.2)
        
        # Score the pedigree
        scorer = SimulatedAnnealingPedigreeOptimizer(ga_pedigree, self.ibd_data, self.individuals_metadata)
        ga_score = scorer._score_pedigree(ga_pedigree)
        
        # Update best if improved
        if ga_score > self.best_score:
            self.best_pedigree = ga_pedigree.copy()
            self.best_score = ga_score
        
        # Update metrics
        metrics['ga_best_pedigree'] = {
            'score': ga_score,
            'num_relationships': ga_pedigree.graph.number_of_edges()
        }
        metrics['score_progression'].append({
            'step': 'genetic_algorithm',
            'parameter': ga_generations,
            'score': ga_score
        })
        
        metrics['time_ga'] = time.time() - start_time
        metrics['total_time'] = time.time() - start_total_time
        
        print("\nHybrid optimization complete!")
        print(f"Best score: {self.best_score}")
        
        return self.best_pedigree, metrics

# Let's test our hybrid approach on the synthetic data
print("Running hybrid optimization on synthetic data...")
hybrid_optimizer = HybridPedigreeOptimizer(synthetic_segments, synthetic_individuals)

# Run with reduced parameters for demonstration purposes
hybrid_best_pedigree, hybrid_metrics = hybrid_optimizer.optimize(
    greedy_min_certainties=[0.3, 0.5],
    sa_iterations=75,  # Reduced for demonstration
    sa_cooling_rates=[0.01, 0.02],
    ga_population_size=6,  # Reduced for demonstration
    ga_generations=10  # Reduced for demonstration
)

# Display timing information
print("\nExecution times:")
print(f"Greedy phase: {hybrid_metrics['time_greedy']:.2f} seconds")
print(f"Simulated annealing phase: {hybrid_metrics['time_sa']:.2f} seconds")
print(f"Genetic algorithm phase: {hybrid_metrics['time_ga']:.2f} seconds")
print(f"Total time: {hybrid_metrics['total_time']:.2f} seconds")

# Create a DataFrame for the score progression
score_progression_df = pd.DataFrame(hybrid_metrics['score_progression'])
print("\nScore progression:")
display(score_progression_df)

# Plot the score progression
plt.figure(figsize=(12, 6))
colors = {'greedy': 'blue', 'simulated_annealing': 'green', 'genetic_algorithm': 'orange'}
markers = {'greedy': 'o', 'simulated_annealing': 's', 'genetic_algorithm': '^'}

for step in score_progression_df['step'].unique():
    step_data = score_progression_df[score_progression_df['step'] == step]
    plt.scatter(
        range(len(step_data)), 
        step_data['score'], 
        label=step.replace('_', ' ').title(),
        color=colors.get(step, 'gray'),
        marker=markers.get(step, 'x'),
        s=100
    )

plt.plot(range(len(score_progression_df)), score_progression_df['score'], 'k-', alpha=0.3)
plt.axhline(y=hybrid_optimizer.best_score, color='r', linestyle='--', alpha=0.7, label='Best Score')

plt.xlabel('Optimization Step')
plt.ylabel('Score')
plt.title('Score Progression During Hybrid Optimization')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# Visualize the final pedigree
plt.figure(figsize=(12, 8))
hybrid_best_pedigree.visualize()
plt.title('Best Pedigree from Hybrid Approach')
plt.show()

# Compare the hybrid approach with the individual methods
print("\nFinal comparison:")
final_comparison = pd.DataFrame([
    {
        'Method': 'Greedy',
        'Score': optimizer._score_pedigree(initial_pedigree),
        'Num Relationships': initial_pedigree.graph.number_of_edges()
    },
    {
        'Method': 'Simulated Annealing',
        'Score': optimizer._score_pedigree(best_pedigree),
        'Num Relationships': best_pedigree.graph.number_of_edges()
    },
    {
        'Method': 'Genetic Algorithm',
        'Score': optimizer._score_pedigree(ga_best_pedigree),
        'Num Relationships': ga_best_pedigree.graph.number_of_edges()
    },
    {
        'Method': 'Hybrid Approach',
        'Score': optimizer._score_pedigree(hybrid_best_pedigree),
        'Num Relationships': hybrid_best_pedigree.graph.number_of_edges()
    }
])

display(final_comparison)

# Plot the final comparison
plt.figure(figsize=(10, 6))
final_comparison.set_index('Method')['Score'].plot(kind='bar', color=['blue', 'green', 'orange', 'red'])
plt.title('Final Score Comparison')
plt.ylabel('Score')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

<cell_type>markdown</cell_type>## 4. Practical Considerations for Pedigree Optimization

When applying these optimization techniques to real-world pedigree reconstruction problems, several practical considerations become important:

<cell_type>markdown</cell_type>### 4.1 Balancing Computational Resources

Pedigree optimization can be computationally intensive. Some strategies to manage resources:

1. **Divide and conquer**: Split large pedigrees into smaller connected components
2. **Incremental optimization**: Start with a subset of individuals, then grow the pedigree
3. **Time budgeting**: Allocate computation time proportionally to pedigree size and complexity
4. **Parallelization**: Run multiple optimization instances in parallel (especially for GA)
5. **Early stopping**: Terminate optimization when improvements plateau

Let's implement a simple utility to analyze the computational complexity of pedigree optimization:

In [ ]:
# Analyze computational complexity of pedigree optimization
def analyze_optimization_complexity(num_individuals_range):
    """
    Analyze the computational complexity of different optimization methods.
    
    Args:
        num_individuals_range: Range of numbers of individuals to analyze
        
    Returns:
        DataFrame with complexity metrics
    """
    complexity_metrics = []
    
    for num_individuals in num_individuals_range:
        # Calculate theoretical complexity metrics
        
        # Search space size
        search_space_size = 2 ** (num_individuals * (num_individuals - 1))
        
        # Greedy algorithm complexity
        # O(n^2) for considering all pairs, with additional factor for relationship determination
        greedy_complexity = num_individuals**2 * math.log(num_individuals)
        
        # Simulated annealing complexity
        # O(iterations * move_cost), where move_cost is typically O(log n) for graph operations
        sa_iterations = 1000  # typical number
        sa_complexity = sa_iterations * num_individuals * math.log(num_individuals)
        
        # Genetic algorithm complexity
        # O(generations * population_size * (crossover_cost + mutation_cost + evaluation_cost))
        ga_generations = 50  # typical number
        ga_population = 20  # typical size
        ga_evaluation_cost = num_individuals**2 * math.log(num_individuals)  # pedigree scoring
        ga_complexity = ga_generations * ga_population * ga_evaluation_cost
        
        # Store metrics
        complexity_metrics.append({
            'Num Individuals': num_individuals,
            'Search Space Size': search_space_size,
            'Greedy Complexity': greedy_complexity,
            'SA Complexity': sa_complexity,
            'GA Complexity': ga_complexity,
            'Hybrid Complexity': greedy_complexity + sa_complexity + ga_complexity
        })
    
    return pd.DataFrame(complexity_metrics)

# Run the analysis for different numbers of individuals
complexity_df = analyze_optimization_complexity(range(5, 21, 5))

# Format large numbers for display
complexity_df['Search Space Size'] = complexity_df['Search Space Size'].apply(lambda x: f"{x:.2e}")

# Display the results
print("Computational Complexity Analysis:")
display(complexity_df)

# Plot the complexity growth
plt.figure(figsize=(12, 6))

complexity_plot_df = analyze_optimization_complexity(range(5, 51, 5))
plt.semilogy(complexity_plot_df['Num Individuals'], complexity_plot_df['Greedy Complexity'], 'b-', label='Greedy')
plt.semilogy(complexity_plot_df['Num Individuals'], complexity_plot_df['SA Complexity'], 'g-', label='Simulated Annealing')
plt.semilogy(complexity_plot_df['Num Individuals'], complexity_plot_df['GA Complexity'], 'r-', label='Genetic Algorithm')

plt.xlabel('Number of Individuals')
plt.ylabel('Computational Complexity (log scale)')
plt.title('Scaling of Computational Complexity with Problem Size')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Function to estimate runtime based on problem size
def estimate_runtime(num_individuals, base_runtime, base_individuals):
    """
    Estimate runtime for a given number of individuals based on a reference runtime.
    
    Args:
        num_individuals: Number of individuals in the target problem
        base_runtime: Runtime for the base problem (in seconds)
        base_individuals: Number of individuals in the base problem
        
    Returns:
        Estimated runtime in seconds
    """
    # We'll use a complexity of O(n^2 * log(n)) as a rough approximation
    complexity_ratio = (num_individuals**2 * math.log(num_individuals)) / (base_individuals**2 * math.log(base_individuals))
    estimated_runtime = base_runtime * complexity_ratio
    return estimated_runtime

# Base runtime from our hybrid optimization
base_runtime = hybrid_metrics['total_time']
base_individuals = len(synthetic_individuals)

# Estimate runtime for larger problems
runtime_estimates = []
for n in [10, 20, 50, 100, 200, 500, 1000]:
    runtime_sec = estimate_runtime(n, base_runtime, base_individuals)
    
    # Convert to appropriate time units
    if runtime_sec < 60:
        runtime_str = f"{runtime_sec:.2f} seconds"
    elif runtime_sec < 3600:
        runtime_str = f"{runtime_sec/60:.2f} minutes"
    elif runtime_sec < 86400:
        runtime_str = f"{runtime_sec/3600:.2f} hours"
    else:
        runtime_str = f"{runtime_sec/86400:.2f} days"
    
    runtime_estimates.append({
        'Num Individuals': n,
        'Estimated Runtime': runtime_str,
        'Runtime (seconds)': runtime_sec
    })

# Display runtime estimates
print("\nEstimated Runtimes:")
display(pd.DataFrame(runtime_estimates))

# Plot the runtime estimates
plt.figure(figsize=(10, 6))
runtime_df = pd.DataFrame(runtime_estimates)
plt.semilogy(runtime_df['Num Individuals'], runtime_df['Runtime (seconds)'], 'o-', color='purple')
plt.axhline(y=60, color='green', linestyle='--', alpha=0.7, label='1 minute')
plt.axhline(y=3600, color='orange', linestyle='--', alpha=0.7, label='1 hour')
plt.axhline(y=86400, color='red', linestyle='--', alpha=0.7, label='1 day')

plt.xlabel('Number of Individuals')
plt.ylabel('Estimated Runtime (seconds, log scale)')
plt.title('Estimated Runtime vs. Problem Size')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

<cell_type>markdown</cell_type>### 4.2 Managing Uncertainty and Ambiguity

Real-world genetic data often contains uncertainty and ambiguity that affects pedigree reconstruction:

1. **Measurement errors**: IBD segments with incorrect boundaries or false positives
2. **Missing individuals**: Pedigrees with "phantom" ancestors not in the dataset
3. **Relationship ambiguity**: Multiple relationship types with similar IBD patterns
4. **Endogamy**: Population groups with high levels of relatedness that complicate inference

Let's implement a utility to analyze the robustness of our optimization algorithms to uncertainty:

In [ ]:
# Analyze robustness to uncertainty
def test_robustness_to_uncertainty(ibd_data, individuals_metadata, noise_levels=None):
    """
    Test the robustness of optimization algorithms to different levels of noise.
    
    Args:
        ibd_data: Original IBD segment data
        individuals_metadata: Metadata about individuals
        noise_levels: List of noise levels to test (standard deviation as fraction of mean)
        
    Returns:
        DataFrame with robustness metrics
    """
    if noise_levels is None:
        noise_levels = [0.0, 0.05, 0.1, 0.2, 0.3]
    
    robustness_metrics = []
    
    # Get baseline results
    print("Computing baseline results...")
    greedy_builder = GreedyPedigreeBuilder(ibd_data, individuals_metadata)
    baseline_pedigree, _ = greedy_builder.build_pedigree(min_certainty=0.3)
    scorer = SimulatedAnnealingPedigreeOptimizer(baseline_pedigree, ibd_data, individuals_metadata)
    baseline_score = scorer._score_pedigree(baseline_pedigree)
    
    # For each noise level
    for noise_level in noise_levels:
        print(f"\nTesting noise level: {noise_level}")
        
        # Create noisy IBD data by perturbing segment lengths
        noisy_ibd = ibd_data.copy()
        
        if noise_level > 0:
            # Add noise to segment lengths
            mean_length = noisy_ibd['gen_seg_len'].mean()
            std_dev = noise_level * mean_length
            noise = np.random.normal(0, std_dev, len(noisy_ibd))
            noisy_ibd['gen_seg_len'] = noisy_ibd['gen_seg_len'] + noise
            
            # Ensure segment lengths remain positive
            noisy_ibd['gen_seg_len'] = noisy_ibd['gen_seg_len'].clip(lower=1)
        
        # Run greedy algorithm on noisy data
        print("  Running greedy algorithm...")
        greedy_builder = GreedyPedigreeBuilder(noisy_ibd, individuals_metadata)
        greedy_pedigree, _ = greedy_builder.build_pedigree(min_certainty=0.3)
        
        # Score the greedy pedigree against the original data (to measure robustness)
        greedy_score_on_original = scorer._score_pedigree(greedy_pedigree)
        greedy_score_ratio = greedy_score_on_original / baseline_score
        
        # Run simulated annealing on noisy data
        print("  Running simulated annealing...")
        sa_optimizer = SimulatedAnnealingPedigreeOptimizer(greedy_pedigree, noisy_ibd, individuals_metadata)
        sa_pedigree, _ = sa_optimizer.optimize(
            max_iterations=100,  # Reduced for demonstration
            initial_temp=10.0,
            cooling_rate=0.01,
            min_temp=0.1
        )
        
        # Score the SA pedigree against the original data
        sa_score_on_original = scorer._score_pedigree(sa_pedigree)
        sa_score_ratio = sa_score_on_original / baseline_score
        
        # Store metrics
        robustness_metrics.append({
            'Noise Level': noise_level,
            'Greedy Score': greedy_score_on_original,
            'Greedy Score Ratio': greedy_score_ratio,
            'SA Score': sa_score_on_original,
            'SA Score Ratio': sa_score_ratio
        })
    
    return pd.DataFrame(robustness_metrics)

# Set a random seed for reproducibility
np.random.seed(42)

# Test robustness with lower noise levels for demonstration
print("Testing robustness to uncertainty...")
robustness_df = test_robustness_to_uncertainty(
    synthetic_segments, synthetic_individuals,
    noise_levels=[0.0, 0.05, 0.1, 0.15, 0.2]
)

# Display the results
print("\nRobustness to Uncertainty:")
display(robustness_df)

# Plot the robustness results
plt.figure(figsize=(10, 6))

plt.plot(robustness_df['Noise Level'], robustness_df['Greedy Score Ratio'], 'b-o', label='Greedy Algorithm')
plt.plot(robustness_df['Noise Level'], robustness_df['SA Score Ratio'], 'g-o', label='Simulated Annealing')

plt.xlabel('Noise Level (fraction of mean)')
plt.ylabel('Score Ratio (noisy/baseline)')
plt.title('Robustness to Noise in IBD Data')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Let's also look at relationship identification consistency
def analyze_relationship_consistency(pedigree1, pedigree2):
    """
    Analyze the consistency of relationships between two pedigrees.
    
    Args:
        pedigree1: First pedigree
        pedigree2: Second pedigree
        
    Returns:
        dict: Consistency metrics
    """
    # Get all edges from both pedigrees
    edges1 = set(pedigree1.graph.edges())
    edges2 = set(pedigree2.graph.edges())
    
    # Calculate consistency metrics
    common_edges = edges1.intersection(edges2)
    only_in_1 = edges1 - edges2
    only_in_2 = edges2 - edges1
    
    # Calculate consistency scores
    if len(edges1) > 0 and len(edges2) > 0:
        jaccard = len(common_edges) / len(edges1.union(edges2))
        precision = len(common_edges) / len(edges1) if len(edges1) > 0 else 0
        recall = len(common_edges) / len(edges2) if len(edges2) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    else:
        jaccard = precision = recall = f1 = 0
    
    return {
        'Common Edges': len(common_edges),
        'Only in First': len(only_in_1),
        'Only in Second': len(only_in_2),
        'Jaccard Similarity': jaccard,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }

# Compare relationship consistency across noise levels
print("\nAnalyzing relationship consistency across noise levels...")
consistency_metrics = []

for i in range(len(robustness_df) - 1):
    noise_level = robustness_df.iloc[i+1]['Noise Level']
    
    # Run optimization on original and noisy data
    print(f"  Testing noise level: {noise_level}")
    
    # Create noisy IBD data
    noisy_ibd = synthetic_segments.copy()
    mean_length = noisy_ibd['gen_seg_len'].mean()
    std_dev = noise_level * mean_length
    noise = np.random.normal(0, std_dev, len(noisy_ibd))
    noisy_ibd['gen_seg_len'] = noisy_ibd['gen_seg_len'] + noise
    noisy_ibd['gen_seg_len'] = noisy_ibd['gen_seg_len'].clip(lower=1)
    
    # Run greedy on original data
    greedy_builder = GreedyPedigreeBuilder(synthetic_segments, synthetic_individuals)
    original_pedigree, _ = greedy_builder.build_pedigree(min_certainty=0.3)
    
    # Run greedy on noisy data
    greedy_builder = GreedyPedigreeBuilder(noisy_ibd, synthetic_individuals)
    noisy_pedigree, _ = greedy_builder.build_pedigree(min_certainty=0.3)
    
    # Analyze consistency
    consistency = analyze_relationship_consistency(original_pedigree, noisy_pedigree)
    consistency['Noise Level'] = noise_level
    
    consistency_metrics.append(consistency)

# Create DataFrame
consistency_df = pd.DataFrame(consistency_metrics)

# Display the results
print("\nRelationship Consistency Analysis:")
display(consistency_df)

# Plot the consistency metrics
plt.figure(figsize=(10, 6))

plt.plot(consistency_df['Noise Level'], consistency_df['Jaccard Similarity'], 'b-o', label='Jaccard Similarity')
plt.plot(consistency_df['Noise Level'], consistency_df['Precision'], 'g-o', label='Precision')
plt.plot(consistency_df['Noise Level'], consistency_df['Recall'], 'r-o', label='Recall')
plt.plot(consistency_df['Noise Level'], consistency_df['F1 Score'], 'k-o', label='F1 Score')

plt.xlabel('Noise Level (fraction of mean)')
plt.ylabel('Consistency Metric')
plt.title('Relationship Consistency with Increasing Noise')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

<cell_type>markdown</cell_type>## 5. Summary and Best Practices

In this lab, we've explored the optimization algorithms that power Bonsai's pedigree reconstruction capabilities. We've implemented and evaluated three key approaches:

1. **Greedy Incremental Construction**: A fast, straightforward approach that builds pedigrees by adding the most confident relationships first
2. **Simulated Annealing**: A probabilistic technique that can escape local optima by temporarily accepting worse solutions
3. **Genetic Algorithms**: A population-based method that evolves multiple candidate pedigrees in parallel

We've also implemented a hybrid approach that combines the strengths of these methods, and analyzed practical considerations like computational complexity and robustness to uncertainty.

### Key Insights

1. The pedigree reconstruction optimization problem is extremely challenging:
   - Vast search space that grows exponentially with the number of individuals
   - Multiple local optima and complex constraints
   - Uncertain and ambiguous data

2. Each optimization approach has different strengths and weaknesses:
   - Greedy algorithms are fast but get stuck in local optima
   - Simulated annealing can escape local optima but requires parameter tuning
   - Genetic algorithms explore diverse solutions but are computationally intensive

3. Hybrid approaches tend to perform best by leveraging the strengths of each method:
   - Starting with greedy construction provides a good initial solution
   - Refining with simulated annealing improves local structure
   - Using genetic algorithms explores alternative global structures

### Best Practices

1. **Data Preparation**:
   - Carefully preprocess IBD data to filter spurious segments
   - Incorporate demographic information when available
   - Identify and handle endogamous populations separately

2. **Algorithm Selection**:
   - For small pedigrees (< 20 individuals): Greedy construction followed by simulated annealing
   - For medium pedigrees (20-100 individuals): Hybrid approach with limited genetic algorithm generations
   - For large pedigrees (> 100 individuals): Divide and conquer or incremental optimization

3. **Parameter Tuning**:
   - Use multiple certainty thresholds for greedy construction
   - Experiment with different cooling schedules for simulated annealing
   - For genetic algorithms, prioritize population diversity over size

4. **Evaluation**:
   - Score against holdout data when possible
   - Consider robustness to noise and missing data
   - Validate with known relationships or genealogical records

### Next Steps

In the next lab, we'll explore how to apply these optimization techniques to real-world genetic genealogy datasets and evaluate the quality of the reconstructed pedigrees.

<cell_type>markdown</cell_type>## 6. Exercises

1. **Parameter Sensitivity Analysis**:
   - Experiment with different parameter settings for simulated annealing (initial temperature, cooling rate)
   - Analyze how these parameters affect the quality of the reconstructed pedigree
   - Plot the relationship between parameter values and pedigree score

2. **Custom Move Operators**:
   - Implement additional move operators for simulated annealing (e.g., subtree relocation, relationship reversal)
   - Compare the effectiveness of different move operators on pedigree reconstruction
   - Design a move selection strategy that adapts based on the current pedigree state

3. **Constraint Handling**:
   - Modify the scoring function to incorporate additional constraints (e.g., maximum number of children, age constraints)
   - Implement a constraint satisfaction approach that guarantees valid pedigrees
   - Compare hard constraint enforcement vs. soft penalty approaches

4. **Performance Optimization**:
   - Profile the execution time of different components of the optimization process
   - Implement parallelization for the genetic algorithm (e.g., parallel fitness evaluation)
   - Design a caching strategy to avoid redundant computation of relationship likelihoods

5. **Advanced Hybrid Approaches**:
   - Design and implement a different hybrid optimization strategy
   - Compare its performance against the approaches presented in this lab
   - Analyze the tradeoffs between computation time and solution quality

<cell_type>markdown</cell_type>## 7. References and Further Reading

1. **Pedigree Reconstruction**
   - Koller, D., & Friedman, N. (2009). *Probabilistic Graphical Models: Principles and Techniques*. MIT Press.
   - Kirkpatrick, B., Ge, S., & Wang, L. (2019). Advances in computational methods for identifying relatives from genetic data. *Nature Reviews Genetics*, 20(5), 273-284.

2. **Optimization Algorithms**
   - Russell, S. J., & Norvig, P. (2016). *Artificial Intelligence: A Modern Approach*. Pearson.
   - Kirkpatrick, S., Gelatt, C. D., & Vecchi, M. P. (1983). Optimization by simulated annealing. *Science*, 220(4598), 671-680.
   - Mitchell, M. (1998). *An Introduction to Genetic Algorithms*. MIT Press.

3. **Genetic Genealogy**
   - Browning, S. R., & Browning, B. L. (2012). Identity by descent between distant relatives: detection and applications. *Annual Review of Genetics*, 46, 617-633.
   - Speed, D., & Balding, D. J. (2015). Relatedness in the post-genomic era: is it still useful? *Nature Reviews Genetics*, 16(1), 33-44.

4. **Software and Tools**
   - Staples, J., et al. (2016). PRIMUS: Rapid reconstruction of pedigrees from genome-wide estimates of identity by descent. *American Journal of Human Genetics*, 98(6), 1193-1204.
   - Ko, A., & Nielsen, R. (2017). Composite likelihood method for inferring local pedigrees. *PLoS Genetics*, 13(8), e1006963.

5. **Online Resources**
   - Bonsai Algorithm Documentation: [https://myheritage.github.io/bonsai](https://github.com/MyHeritage/bonsai)
   - ISOGG Wiki - Identity by Descent: [https://isogg.org/wiki/Identity_by_descent](https://isogg.org/wiki/Identity_by_descent)
   - Genetic Genealogy Tools: [https://geneticgenealogist.net/](https://thegeneticgenealogist.com/)